In [4]:
import json
import os
import pandas as pd

In [5]:
DATA_DIR = r"C:\Users\Adnan Khader\Projects\cricket\t20s_json"

In [6]:
matches = []
deliveries = []

In [7]:
deliveries = []
matches = []

for file in os.listdir(DATA_DIR):
    if not file.endswith(".json"):
        continue
    
    match_id = file.replace(".json", "")
    
    with open(os.path.join(DATA_DIR, file), "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # -------------------------
    # MATCH-LEVEL DATA
    # -------------------------
    info = data.get("info", {})
    teams = info.get("teams", [])
    team_1 = teams[0] if len(teams) > 0 else None
    team_2 = teams[1] if len(teams) > 1 else None
    
    match_row = {
        "match_id": match_id,
        "date": info.get("dates", [None])[0],
        "venue": info.get("venue"),
        "city": info.get("city"),
        "gender": info.get("gender"),
        "match_type": info.get("match_type"),
        "team_type": info.get("team_type"),
        "team_1": team_1,
        "team_2": team_2,
        "toss_winner": info.get("toss", {}).get("winner"),
        "toss_decision": info.get("toss", {}).get("decision"),
        "winner": info.get("outcome", {}).get("winner")
    }
    matches.append(match_row)
    
    # -------------------------
    # BALL-BY-BALL DATA (ONLY INNINGS 1 & 2)
    # -------------------------
    innings_list = data.get("innings", [])
    
    # CRITICAL FIX: Only process first 2 innings
    for innings_index, innings in enumerate(innings_list[:2], start=1):  # ← FIXED: [:2] limits to first 2 innings
        batting_team = innings.get("team")
        
        # Determine bowling team
        if batting_team == team_1:
            bowling_team = team_2
        elif batting_team == team_2:
            bowling_team = team_1
        else:
            bowling_team = None
        
        overs_list = innings.get("overs", [])
        
        if not isinstance(overs_list, list):
            continue
        
        for over_data in overs_list:
            over_number = over_data.get("over")
            deliveries_list = over_data.get("deliveries", [])
            
            for ball_index, ball_data in enumerate(deliveries_list, start=1):
                delivery_row = {
                    "match_id": match_id,
                    "innings_id": innings_index,          # Now guaranteed to be only 1 or 2
                    "batting_team": batting_team,
                    "bowling_team": bowling_team,
                    "over": over_number,
                    "ball": ball_index,
                    "batter": ball_data.get("batter"),
                    "bowler": ball_data.get("bowler"),
                    "non_striker": ball_data.get("non_striker"),
                    "runs_batter": ball_data.get("runs", {}).get("batter", 0),
                    "runs_extras": ball_data.get("runs", {}).get("extras", 0),
                    "runs_total": ball_data.get("runs", {}).get("total", 0),
                    "is_wicket": 1 if "wickets" in ball_data else 0
                }
                deliveries.append(delivery_row)

In [8]:
matches_df = pd.DataFrame(matches)
deliveries_df = pd.DataFrame(deliveries)


In [9]:
len(matches_df)

4972

In [10]:
matches_df.head()

,match_id,date,venue,city,gender,match_type,team_type,team_1,team_2,toss_winner,toss_decision,winner
0,1001349,2017-02-17,Melbourne Cricket Ground,None,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka
1,1001351,2017-02-19,"Simonds Stadium, South Geelong",Victoria,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka
2,1001353,2017-02-22,Adelaide Oval,None,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Australia
3,1004729,2016-09-05,"Bready Cricket Club, Magheramason",Londonderry,male,T20,international,Ireland,Hong Kong,Hong Kong,bat,Hong Kong
4,1007655,2016-06-18,Harare Sports Club,None,male,T20,international,Zimbabwe,India,India,field,Zimbabwe


In [11]:
matches_df['date'].max()

'2026-02-04'

In [12]:
deliveries_df.head()

,match_id,innings_id,batting_team,bowling_team,over,ball,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,is_wicket
0,1001349,1,Australia,Sri Lanka,0,1,AJ Finch,SL Malinga,M Klinger,0,0,0,0
1,1001349,1,Australia,Sri Lanka,0,2,AJ Finch,SL Malinga,M Klinger,0,0,0,0
2,1001349,1,Australia,Sri Lanka,0,3,AJ Finch,SL Malinga,M Klinger,1,0,1,0
3,1001349,1,Australia,Sri Lanka,0,4,M Klinger,SL Malinga,AJ Finch,2,0,2,0
4,1001349,1,Australia,Sri Lanka,0,5,M Klinger,SL Malinga,AJ Finch,0,0,0,0


In [13]:
deliveries_df['innings_id'].value_counts()

innings_id
1    606557
2    517452
Name: count, dtype: int64

In [14]:
deliveries_df = deliveries_df.merge(
    matches_df[['match_id', 'team_1', 'team_2']],
    on='match_id',
    how='left'
)


In [15]:
deliveries_df['bowling_team'] = deliveries_df.apply(
    lambda row: row['team_2']
    if row['batting_team'] == row['team_1']
    else row['team_1'],
    axis=1
)


In [16]:
deliveries_df[deliveries_df['batting_team']=='India'].head()

,match_id,innings_id,batting_team,bowling_team,over,ball,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,is_wicket,team_1,team_2
1110,1007655,2,India,Zimbabwe,0,1,KL Rahul,DT Tiripano,Mandeep Singh,0,0,0,1,Zimbabwe,India
1111,1007655,2,India,Zimbabwe,0,2,AT Rayudu,DT Tiripano,Mandeep Singh,0,0,0,0,Zimbabwe,India
1112,1007655,2,India,Zimbabwe,0,3,AT Rayudu,DT Tiripano,Mandeep Singh,0,0,0,0,Zimbabwe,India
1113,1007655,2,India,Zimbabwe,0,4,AT Rayudu,DT Tiripano,Mandeep Singh,4,0,4,0,Zimbabwe,India
1114,1007655,2,India,Zimbabwe,0,5,AT Rayudu,DT Tiripano,Mandeep Singh,0,0,0,0,Zimbabwe,India


In [17]:
len(deliveries_df)

1124009

In [18]:
deliveries_df['batting_team'].unique()

array(['Australia', 'Sri Lanka', 'Hong Kong', 'Ireland', 'Zimbabwe',
       'India', 'Bangladesh', 'New Zealand', 'South Africa', 'England',
       'West Indies', 'Pakistan', 'Scotland', 'Oman', 'Netherlands',
       'United Arab Emirates', 'Papua New Guinea', 'ICC World XI',
       'Thailand', 'Uganda', 'Malaysia', 'Botswana', 'Lesotho', 'Malawi',
       'Namibia', 'Sierra Leone', 'Mozambique', 'Nepal', 'China',
       'Kuwait', 'Philippines', 'Vanuatu', 'United States of America',
       'Germany', 'Italy', 'Nigeria', 'Tanzania', 'Rwanda', 'Kenya',
       'Japan', 'Indonesia', 'Fiji', 'Samoa', 'Canada', 'Ghana',
       'Guernsey', 'Denmark', 'Norway', 'Jersey', 'Maldives', 'Mali',
       'Singapore', 'Qatar', 'South Korea', 'Bermuda', 'Cayman Islands',
       'Portugal', 'Spain', 'Gibraltar', 'Bhutan', 'Saudi Arabia',
       'Bahrain', 'Iran', 'Austria', 'Belgium', 'Luxembourg',
       'Czech Republic', 'Isle of Man', 'Bulgaria', 'Romania', 'Greece',
       'Serbia', 'Malta', 'France

In [19]:
t20_world_cup_2026_teams = [
    "Afghanistan", 
    "Australia", 
    "Canada", 
    "England", 
    "India", 
    "Ireland", 
    "Italy", 
    "Namibia", 
    "Nepal", 
    "Netherlands", 
    "New Zealand", 
    "Oman", 
    "Pakistan", 
    "Scotland", 
    "South Africa", 
    "Sri Lanka", 
    "United Arab Emirates", 
    "United States of America", 
    "West Indies", 
    "Zimbabwe"
]

In [20]:
for i in t20_world_cup_2026_teams:
    if i  not in deliveries_df['batting_team'].unique():
        print(i)

Afghanistan


In [21]:
import pandas as pd

matches_df["date"] = pd.to_datetime(matches_df["date"])

cutoff_date = matches_df["date"].max() - pd.DateOffset(years=2)

valid_matches = matches_df[
    (matches_df["gender"] == "male") &
    (matches_df["match_type"] == "T20") &
    (matches_df["team_type"] == "international") &
    (matches_df["date"] >= cutoff_date) 
    
]

valid_match_ids = set(valid_matches["match_id"])


In [22]:
deliveries_valid = deliveries_df[
    deliveries_df["match_id"].isin(valid_match_ids)
].copy()


In [23]:
deliveries_valid.head(5)

,match_id,innings_id,batting_team,bowling_team,over,ball,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,is_wicket,team_1,team_2
431841,1375850,1,Australia,West Indies,0,1,DA Warner,AJ Hosein,JP Inglis,0,0,0,0,Australia,West Indies
431842,1375850,1,Australia,West Indies,0,2,DA Warner,AJ Hosein,JP Inglis,0,0,0,0,Australia,West Indies
431843,1375850,1,Australia,West Indies,0,3,DA Warner,AJ Hosein,JP Inglis,4,0,4,0,Australia,West Indies
431844,1375850,1,Australia,West Indies,0,4,DA Warner,AJ Hosein,JP Inglis,1,0,1,0,Australia,West Indies
431845,1375850,1,Australia,West Indies,0,5,JP Inglis,AJ Hosein,DA Warner,2,0,2,0,Australia,West Indies


In [24]:
batting_stats = (
    deliveries_valid
    .groupby(["batter", "batting_team"])
    .agg(
        balls_faced=("batter", "count"),
        runs=("runs_batter", "sum"),
        matches_played=("match_id", "nunique"),

        # NEW COUNTS
        dismissals=("is_wicket", "sum"),
        dot_balls=("runs_total", lambda x: (x == 0).sum()),
        fours=("runs_batter", lambda x: (x == 4).sum()),
        sixes=("runs_batter", lambda x: (x == 6).sum()),
    )
    .reset_index()
)



In [25]:
# Strike rate 
batting_stats["strike_rate"] = (
    batting_stats["runs"] / batting_stats["balls_faced"] * 100
)

# Batting average
batting_stats["batting_average"] = (
    batting_stats["runs"] / batting_stats["dismissals"]
)

# Dot ball percentage
batting_stats["dot_ball_pct"] = (
    batting_stats["dot_balls"] / batting_stats["balls_faced"]
)

# Boundary percentage
batting_stats["boundary_pct"] = (
    (batting_stats["fours"] + batting_stats["sixes"]) / batting_stats["balls_faced"]
)


In [26]:
### Deriving Batting Position:
#  weighted by frequency
# Calculate usual batting position for each player

batting_order = (
    deliveries_valid
    .sort_values(['match_id', 'innings_id', 'over', 'ball'])
    .drop_duplicates(subset=['match_id', 'innings_id', 'batter'], keep='first')
    .groupby(['match_id', 'innings_id'])
    .apply(lambda x: x.assign(batting_position=range(1, len(x) + 1)))
    .reset_index(drop=True)
)

usual_position = (
    batting_order
    .groupby('batter')['batting_position']
    .agg([
        ('usual_batting_position', lambda x: int(x.mode()[0])),  # Most frequent
        ('times_batted', 'count'),
        ('position_std', 'std')  # Shows consistency
    ])
    .reset_index()
)

# Merge with batting_stats
batting_stats = batting_stats.merge(
    usual_position[['batter', 'usual_batting_position']], 
    on='batter', 
    how='left'
)

# Handle players with missing positions (didn't bat enough)
batting_stats['usual_batting_position'] = batting_stats['usual_batting_position'].fillna(11).astype(int)



C:\Users\Adnan Khader\AppData\Local\Temp\ipykernel_9804\3128596900.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(batting_position=range(1, len(x) + 1)))


### Add phase-wise batting strike rates to batting_stats

Definitions:
- Powerplay (pp): overs 0–5
- Middle overs (middle): overs 6–14
- Death overs (death): overs 15–19

In [27]:
def get_phase(over):
    if over <= 6:
        return "pp"
    elif over <= 14:
        return "middle"
    else:
        return "death"

deliveries_valid["phase"] = deliveries_valid["over"].apply(get_phase)

In [28]:
phase_batting = (
    deliveries_valid
    .groupby(["batter", "batting_team", "phase"])
    .agg(
        balls_faced=("batter", "count"),
        runs_scored=("runs_batter", "sum")
    )
    .reset_index()
)

In [29]:
phase_batting["strike_rate"] = (
    phase_batting["runs_scored"] / phase_batting["balls_faced"] * 100
)

In [30]:
phase_strike_rates = (
    phase_batting
    .pivot(
        index=["batter", "batting_team"],
        columns="phase",
        values="strike_rate"
    )
    .reset_index()
)

phase_strike_rates.columns.name = None

phase_strike_rates = phase_strike_rates.rename(columns={
    "pp": "pp_strike_rate",
    "middle": "middle_strike_rate",
    "death": "death_strike_rate"
})

In [31]:
bowling_stats = (
    deliveries_valid
    .groupby(["bowler","bowling_team"])
    .agg(
        balls_bowled=("bowler", "count"),
        runs_conceded=("runs_total", "sum"),
        matches_played=("match_id", "nunique"),
        dot_balls=("runs_total", lambda x: (x == 0).sum()),
        wickets=("is_wicket","sum")
    )
    .reset_index()
)

bowling_stats["overs_bowled"] = bowling_stats["balls_bowled"] / 6
bowling_stats["economy"] = bowling_stats["runs_conceded"] / bowling_stats["overs_bowled"]
bowling_stats["dot_ball_pct"] = bowling_stats["dot_balls"] / bowling_stats["balls_bowled"]


In [32]:
batting_stats = batting_stats.merge(
    phase_strike_rates,
    on=["batter", "batting_team"],
    how="left"
)


# ---------------------------------------------------------
# Handle players who did not bat in some phases
# ---------------------------------------------------------

batting_stats[
    ["pp_strike_rate", "middle_strike_rate", "death_strike_rate"]
] = batting_stats[
    ["pp_strike_rate", "middle_strike_rate", "death_strike_rate"]
].fillna(0)

In [33]:
# Batting Role

def batting_role_from_position(pos):
    if pos <= 2:
        return "Opener"
    elif pos <= 4:
        return "Top order"
    elif pos <= 6:
        return "Middle order"
    elif pos <= 8:
        return "Finisher"
    else:
        return "Tail"


In [34]:
batting_stats["batting_role"] = (
    batting_stats["usual_batting_position"]
    .apply(batting_role_from_position)
)

In [35]:
batting_stats.head()

,batter,batting_team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,usual_batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
0,A Ahmadhel,Bulgaria,6,2,1,2,4,0,0,33.333333,1.000000,0.666667,0.000000,2,0.0,0.000000,33.333333,Opener
1,A Alexander,Indonesia,35,50,8,5,10,2,4,142.857143,10.000000,0.285714,0.171429,7,220.0,76.923077,100.000000,Finisher
2,A Athanaze,West Indies,263,326,14,13,116,31,17,123.954373,25.076923,0.441065,0.182510,1,0.0,118.367347,125.233645,Opener
3,A Augastin,Tanzania,166,170,9,9,72,20,4,102.409639,18.888889,0.433735,0.144578,3,137.5,112.643678,79.365079,Top order
4,A Awasthi,Croatia,55,11,5,4,39,0,0,20.000000,2.750000,0.709091,0.000000,6,25.0,23.809524,13.636364,Middle order


### Batting Stats Completed

In [36]:
"""
Add bowling_average and bowling_strike_rate to bowling_stats
"""

import numpy as np

# Bowling average
bowling_stats["bowling_average"] = (
    bowling_stats["runs_conceded"] / bowling_stats["wickets"]
)

# Bowling strike rate
bowling_stats["bowling_strike_rate"] = (
    bowling_stats["balls_bowled"] / bowling_stats["wickets"]
)

# Handle divide-by-zero cases safely
bowling_stats.replace([np.inf, -np.inf], np.nan, inplace=True)
bowling_stats[["bowling_average", "bowling_strike_rate"]] = (
    bowling_stats[["bowling_average", "bowling_strike_rate"]]
    .fillna(0)
)


In [37]:
"""
Add phase-wise bowling economy to bowling_stats

Definitions:
- Powerplay (pp): overs 0–5
- Middle overs (middle): overs 6–14
- Death overs (death): overs 15–19

Economy = (Runs Conceded / Balls Bowled) * 6

Assumptions:
- deliveries_valid has one row per legal delivery
- deliveries_valid contains:
  bowler, bowling_team, over, runs_total
- bowling_stats already exists with bowler + bowling_team
"""

import numpy as np

# ---------------------------------------------------------
# 1. Assign phase to each delivery
# ---------------------------------------------------------

def get_phase(over):
    if over <= 5:
        return "pp"
    elif over <= 14:
        return "middle"
    else:
        return "death"

deliveries_valid["phase"] = deliveries_valid["over"].apply(get_phase)


# ---------------------------------------------------------
# 2. Aggregate runs conceded & balls bowled by phase
# ---------------------------------------------------------

phase_bowling = (
    deliveries_valid
    .groupby(["bowler", "bowling_team", "phase"])
    .agg(
        balls_bowled=("bowler", "count"),
        runs_conceded=("runs_total", "sum")
    )
    .reset_index()
)


# ---------------------------------------------------------
# 3. Compute economy for each phase
# ---------------------------------------------------------

phase_bowling["economy"] = (
    phase_bowling["runs_conceded"] / phase_bowling["balls_bowled"] * 6
)


# ---------------------------------------------------------
# 4. Pivot phases into columns
# ---------------------------------------------------------

phase_economy = (
    phase_bowling
    .pivot(
        index=["bowler", "bowling_team"],
        columns="phase",
        values="economy"
    )
    .reset_index()
)

phase_economy.columns.name = None

phase_economy = phase_economy.rename(columns={
    "pp": "pp_economy",
    "middle": "middle_economy",
    "death": "death_economy"
})


# ---------------------------------------------------------
# 5. Merge phase-wise economy into bowling_stats
# ---------------------------------------------------------

bowling_stats = bowling_stats.merge(
    phase_economy,
    on=["bowler", "bowling_team"],
    how="left"
)


# ---------------------------------------------------------
# 6. Handle bowlers without balls in some phases
# ---------------------------------------------------------

bowling_stats[
    ["pp_economy", "middle_economy", "death_economy"]
] = bowling_stats[
    ["pp_economy", "middle_economy", "death_economy"]
].replace([np.inf, -np.inf], np.nan).fillna(0)


In [38]:
bowling_stats.head()

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy
0,A Alexander,Indonesia,108,105,8,51,9,18.000000,5.833333,0.472222,11.666667,12.0,5.842105,4.645161,6.769231
1,A Athanaze,West Indies,7,16,1,1,0,1.166667,13.714286,0.142857,0.000000,0.0,0.000000,13.714286,0.000000
2,A Augastin,Tanzania,77,83,5,32,5,12.833333,6.467532,0.415584,16.600000,15.4,5.142857,6.120000,14.000000
3,A Awasthi,Croatia,27,75,2,2,1,4.500000,16.666667,0.074074,75.000000,27.0,16.736842,0.000000,16.500000
4,A Beresford-Peirse,Croatia,202,261,9,73,5,33.666667,7.752475,0.361386,52.200000,40.4,6.461538,8.666667,7.583333


In [39]:
bowling_stats=bowling_stats[bowling_stats["bowling_team"].isin(t20_world_cup_2026_teams)]

In [40]:
batting_stats=batting_stats[batting_stats["batting_team"].isin(t20_world_cup_2026_teams)]

In [41]:
bowling_stats[bowling_stats['bowling_team']=='India']

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy
101,AR Patel,India,545,686,30,215,37,90.833333,7.552294,0.394495,18.540541,14.729730,6.400000,7.216667,9.436364
138,Abhishek Sharma,India,155,225,17,43,6,25.833333,8.709677,0.277419,37.500000,25.833333,10.666667,7.834711,16.285714
201,Arshdeep Singh,India,694,942,30,279,55,115.666667,8.144092,0.402017,17.127273,12.618182,8.610879,5.731343,8.273196
220,Avesh Khan,India,104,137,5,44,9,17.333333,7.903846,0.423077,15.222222,11.555556,9.285714,5.000000,7.440000
334,CV Varun,India,649,807,28,265,59,108.166667,7.460709,0.408320,13.677966,11.000000,6.835443,7.747178,6.850394
537,HH Pandya,India,623,853,34,235,35,103.833333,8.215088,0.377207,24.371429,17.800000,8.416667,8.608696,7.734940
561,Harshit Rana,India,172,300,8,64,9,28.666667,10.465116,0.372093,33.333333,19.111111,10.775510,12.600000,9.612903
680,JJ Bumrah,India,503,579,22,232,33,83.833333,6.906561,0.461233,17.545455,15.242424,7.472393,6.880000,6.566038
781,KK Ahmed,India,93,110,4,42,3,15.500000,7.096774,0.451613,36.666667,31.000000,7.161290,0.000000,7.064516
819,Kuldeep Yadav,India,357,430,16,133,33,59.500000,7.226891,0.372549,13.030303,10.818182,5.454545,7.426710,10.000000


In [42]:
batting_stats[batting_stats['batting_team']=='India'][['batting_role','batter','usual_batting_position']]

,batting_role,batter,usual_batting_position
145,Finisher,AR Patel,8
193,Opener,Abhishek Sharma,1
275,Tail,Arshdeep Singh,9
296,Tail,Avesh Khan,9
461,Tail,CV Varun,10
569,Middle order,Dhruv Jurel,5
752,Middle order,HH Pandya,5
775,Finisher,Harshit Rana,7
847,Opener,Ishan Kishan,2
939,Tail,JJ Bumrah,10


In [43]:
batting_stats[batting_stats['batter'].duplicated(keep=False)]

,batter,batting_team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,usual_batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
2020,Shoaib Khan,Oman,111,96,7,5,63,6,6,86.486486,19.2,0.567568,0.108108,3,50.0,92.857143,76.923077,Top order
2021,Shoaib Khan,United Arab Emirates,18,14,2,2,11,1,1,77.777778,7.0,0.611111,0.111111,3,0.0,42.857143,100.000000,Top order


In [44]:
bowling_stats[bowling_stats['bowler'].duplicated(keep=False)]

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy


In [45]:
batting_stats.sort_values(by='batting_team')['batter'].unique()

array(['MP Kuhnemann', 'AT Carey', 'SH Johnson', 'NT Ellis', 'SPD Smith',
       'MW Short', 'MT Renshaw', 'MS Wade', 'MR Marsh', 'MP Stoinis',
       'MJ Owen', 'AM Hardie', 'MA Starc', 'BJ Dwarshuis', 'JR Hazlewood',
       'JP Inglis', 'J Fraser-McGurk', 'J Edwards', 'C Connolly',
       'C Green', 'TM Head', 'GJ Maxwell', 'TH David', 'JR Philippe',
       'PJ Cummins', 'DA Warner', 'A Zampa', 'SA Abbott', 'XC Bartlett',
       'U Bhagwan', 'Saad Bin Zafar', 'Shivam Sharma', 'Kaleem Sana',
       'Akhil Kumar', 'Pargat Singh', 'Mihir Patel', 'Ali Nadeem',
       'YS Samra', 'Ansh Patel', 'Sukhjinder Singh', 'A Johnson',
       'M Gill', 'Ravinderpal Singh', 'Jaskaran Singh', 'K Tathgur',
       'R Pathan', 'Parveen Kumar', 'NR Kirton', 'NS Dhaliwal',
       'SA Wijeratne', 'S Movva', 'RR Joshi', 'D Heyliger', 'AS Hundal',
       'H Thaker', 'DS Bajwa', 'LS Livingstone', 'BM Duckett',
       'LA Dawson', 'JM Cox', 'BA Carse', 'JC Buttler', 'JL Smith',
       'RJW Topley', 'S Mahmood'

In [46]:
bowling_stats.sort_values(by='bowling_team')['bowler'].unique()

array(['JR Hazlewood', 'PJ Cummins', 'NT Ellis', 'SA Abbott',
       'BJ Dwarshuis', 'SH Johnson', 'C Connolly', 'XC Bartlett',
       'C Green', 'MW Short', 'AM Hardie', 'JP Behrendorff', 'MR Marsh',
       'MP Kuhnemann', 'MJ Owen', 'MP Stoinis', 'AC Agar', 'GJ Maxwell',
       'M Beardman', 'A Zampa', 'RP Meredith', 'MA Starc', 'J Edwards',
       'Ansh Patel', 'JOA Gordon', 'Shahid Ahmadzai', 'U Bhagwan',
       'NR Kirton', 'Jaskaran Singh', 'Junaid Siddiqui', 'Kaleem Sana',
       'Shivam Sharma', 'Sukhjinder Singh', 'Saad Bin Zafar', 'H Thaker',
       'D Heyliger', 'Pargat Singh', 'DS Bajwa', 'N Dutta', 'Akhil Kumar',
       'Ravinderpal Singh', 'A Johnson', 'YS Samra', 'Parveen Kumar',
       'RR Joshi', 'Rehan Ahmed', 'L Wood', 'LA Dawson', 'LS Livingstone',
       'MJ Potts', 'AAP Atkinson', 'CJ Jordan', 'RJW Topley', 'S Baker',
       'MA Wood', 'MM Ali', 'BA Carse', 'SM Curran', 'AU Rashid',
       'DR Mousley', 'WG Jacks', 'J Overton', 'JG Bethell', 'JC Archer',
       'J

In [47]:
name_mapping = {
    # India
    'Suryakumar Yadav': 'Suryakumar Yadav',
    'SA Yadav': 'Suryakumar Yadav',  # Duplicate handling
    'Abhishek Sharma': 'Abhishek Sharma',
    'Tilak Varma': 'Tilak Varma',
    'SV Samson': 'Sanju Samson',
    'S Dube': 'Shivam Dube',
    'Ishan Kishan': 'Ishan Kishan',
    'HH Pandya': 'Hardik Pandya',
    'Arshdeep Singh': 'Arshdeep Singh',
    'JJ Bumrah': 'Jasprit Bumrah',
    'Mohammed Siraj': 'Mohammed Siraj',
    'CV Varun': 'Varun Chakaravarthy',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'AR Patel': 'Axar Patel',
    'Washington Sundar': 'Washington Sundar',
    'RK Singh': 'Rinku Singh',
    
    # USA
    'MD Patel': 'Monank Patel',
    'Jasdeep Singh': 'Jessy Singh',
    'AGS Gous': 'Andries Gous',
    'Shehan Jayasuriya': 'Shehan Jayasuriya',
    'Milind Kumar': 'Milind Kumar',
    'Shayan Jahangir': 'Shayan Jahangir',
    'SR Mukkamalla': 'Saiteja Mukkamala',
    'SP Krishnamurthi': 'Sanjay Krishnamurthi',
    'Harmeet Singh': 'Harmeet Singh',
    'NP Kenjige': 'Nosthush Kenjige',
    'SC van Schalkwyk': 'Shadley Van Schalkwyk',
    'SN Netravalkar': 'Saurabh Netravalkar',
    'Ali Khan': 'Ali Khan',
    'Mohammad Mohsin': 'Mohammad Mohsin',
    'Shubham Ranjane': 'Shubham Ranjane',
    
    # Namibia
    'MG Erasmus': 'Gerhard Erasmus',
    'ZE Green': 'Zane Green',
    'BM Scholtz': 'Bernard Scholtz',
    'R Trumpelmann': 'Ruben Trumpelmann',
    'JJ Smit': 'JJ Smit',
    'JN Frylinck': 'Jan Frylinck',
    'L Steenkamp': 'Louren Steenkamp',
    'MB Kruger': 'Malan Kruger',
    'JN Loftie-Eaton': 'Nicol Loftie-Eaton',
    'JT Brassell': 'Jack Brassell',
    'B Shikongo': 'Ben Shikongo',
    'JC Balt': 'JC Balt',
    'DO Leicher': 'Dylan Leicher',
    'WP Myburgh': 'WP Myburgh',
    'Max Heingo': 'Max Heingo',
    
    
    # Netherlands
    'SA Edwards': 'Scott Edwards',
    'CN Ackermann': 'Colin Ackermann',
    'NRJ Croes': 'Noah Croes',
    'BFW de Leede': 'Bas de Leede',
    'A Dutt': 'Aryan Dutt',
    'Fred Klaassen': 'Fred Klaassen',
    'K Klein': 'Kyle Klein',
    'M Levitt': 'Michael Levitt',
    'ZB Lion-Cachet': 'Zach Lion-Cachet',
    "MP O'Dowd": "Max O'Dowd",
    'LV van Beek': 'Logan van Beek',
    'T van der Gugten': 'Timm van der Gugten',
    'RE van der Merwe': 'Roelof van der Merwe',
    'PA van Meekeren': 'Paul van Meekeren',
    'Saqib Zulfiqar': 'Saqib Zulfiqar',
    
    # Pakistan
    'Agha Salman': 'Salman Ali Agha',
    'Abrar Ahmed': 'Abrar Ahmed',
    'Babar Azam': 'Babar Azam',
    'Faheem Ashraf': 'Faheem Ashraf',
    'Fakhar Zaman': 'Fakhar Zaman',
    'Khawaja Nafay': 'Khawaja Nafay',
    'Mohammad Nawaz': 'Mohammad Nawaz',
    'Salman Mirza': 'Mohammad Salman Mirza',
    'Naseem Shah': 'Naseem Shah',
    'Sahibzada Farhan': 'Sahibzada Farhan',
    'Saim Ayub': 'Saim Ayub',
    'Shaheen Shah Afridi': 'Shaheen Shah Afridi',
    'Shadab Khan': 'Shadab Khan',
    'Usman Khan': 'Usman Khan',
    'Usman Tariq': 'Usman Tariq',
    
    # Australia
    'MR Marsh': 'Mitchell Marsh',
    'XC Bartlett': 'Xavier Bartlett',
    'C Connolly': 'Cooper Connolly',
    'TH David': 'Tim David',
    'BJ Dwarshuis': 'Ben Dwarshuis',
    'C Green': 'Cameron Green',
    'NT Ellis': 'Nathan Ellis',
    'JR Hazlewood': 'Josh Hazlewood',
    'TM Head': 'Travis Head',
    'JP Inglis': 'Josh Inglis',
    'MP Kuhnemann': 'Matthew Kuhnemann',
    'GJ Maxwell': 'Glenn Maxwell',
    'MT Renshaw': 'Matthew Renshaw',
    'MP Stoinis': 'Marcus Stoinis',
    'A Zampa': 'Adam Zampa',
    
    # Sri Lanka
    'MD Shanaka': 'Dasun Shanaka',
    'P Nissanka': 'Pathum Nissanka',
    'K Mishara': 'Kamil Mishara',
    'BKG Mendis': 'Kusal Mendis',
    'PHKD Mendis': 'Kamindu Mendis',
    'MDKJ Perera': 'Kusal Janith Perera',
    'KIC Asalanka': 'Charith Asalanka',
    'J Liyanage': 'Janith Liyanage',
    'P Rathnayake': 'Pavan Rathnayake',
    'Wanindu Hasaranga': 'Wanindu Hasaranga',
    'DN Wellalage': 'Dunith Wellalage',
    'M Theekshana': 'Maheesh Theekshana',
    'PVD Chameera': 'Dushmantha Chameera',
    'M Pathirana': 'Matheesha Pathirana',
    'E Malinga': 'Eshan Malinga',
    
    # Zimbabwe
    'Sikandar Raza': 'Sikandar Raza',
    'BJ Bennett': 'Brian Bennett',
    'RP Burl': 'Ryan Burl',
    'AG Cremer': 'Graeme Cremer',
    'B Evans': 'Bradley Evans',
    'C Madande': 'Clive Madande',
    'TT Maposa': 'Tinotenda Maposa',
    'T Marumani': 'Tadiwanashe Marumani',
    'WP Masakadza': 'Wellington Masakadza',
    'T Munyonga': 'Tony Munyonga',
    'T Musekiwa': 'Tashinga Musekiwa',
    'B Muzarabani': 'Blessing Muzarabani',
    'D Myers': 'Dion Myers',
    'R Ngarava': 'Richard Ngarava',
    'BRM Taylor': 'Brendan Taylor',
    
    # Ireland
    'PR Stirling': 'Paul Stirling',
    'MR Adair': 'Mark Adair',
    'GR Adair': 'Ross Adair',
    'BF Calitz': 'Ben Calitz',
    'C Campher': 'Curtis Campher',
    'GJ Delany': 'Gareth Delany',
    'GH Dockrell': 'George Dockrell',
    'MJ Humphreys': 'Matthew Humphreys',
    'J Little': 'Josh Little',
    'BJ McCarthy': 'Barry McCarthy',
    'HT Tector': 'Harry Tector',
    'TH Tector': 'Tim Tector',
    'L Tucker': 'Lorcan Tucker',
    'B White': 'Ben White',
    'CA Young': 'Craig Young',
    
    # Oman
    'Jatinder Singh': 'Jatinder Singh',
    'V Shukla': 'Vinayak Shukla',
    'Mohammad Nadeem': 'Mohammad Nadeem',
    'Shakeel Ahmed': 'Shakeel Ahmad',
    'Hammad Mirza': 'Hammad Mirza',
    'Wasim Ali': 'Wasim Ali',
    'K Sonavale': 'Karan Sonavale',
    'Shah Faisal': 'Shah Faisal',
    'Nadeem Khan': 'Nadeem Khan',
    'Sufyan Mehmood': 'Sufyan Mehmood',
    'Jay Odedra': 'Jay Odedra',
    'Shafiq Jan': 'Shafiq Jan',
    'AR Odedara': 'Ashish Odedara',
    'J Ramanandi': 'Jiten Ramanandi',
    'Aamir Kaleem': 'Aamir Kaleem',
    
    # South Africa
    'AK Markram': 'Aiden Markram',
    'C Bosch': 'Corbin Bosch',
    'D Brevis': 'Dewald Brevis',
    'Q de Kock': 'Quinton de Kock',
    'M Jansen': 'Marco Jansen',
    'GF Linde': 'George Linde',
    'KA Maharaj': 'Keshav Maharaj',
    'KT Maphaka': 'Kwena Maphaka',
    'DA Miller': 'David Miller',
    'L Ngidi': 'Lungi Ngidi',
    'A Nortje': 'Anrich Nortje',
    'K Rabada': 'Kagiso Rabada',
    'RD Rickelton': 'Ryan Rickelton',
    'JF Smith': 'Jason Smith',
    'T Stubbs': 'Tristan Stubbs',
    
    # New Zealand
    'MJ Santner': 'Mitchell Santner',
    'FH Allen': 'Finn Allen',
    'MG Bracewell': 'Michael Bracewell',
    'MS Chapman': 'Mark Chapman',
    'DP Conway': 'Devon Conway',
    'JA Duffy': 'Jacob Duffy',
    'LH Ferguson': 'Lockie Ferguson',
    'MJ Henry': 'Matt Henry',
    'KA Jamieson': 'Kyle Jamieson',
    'DJ Mitchell': 'Daryl Mitchell',
    'JDS Neesham': 'James Neesham',
    'GD Phillips': 'Glenn Phillips',
    'R Ravindra': 'Rachin Ravindra',
    'TL Seifert': 'Tim Seifert',
    'IS Sodhi': 'Ish Sodhi',
    
    # Canada
    'DS Bajwa': 'Dilpreet Bajwa',
    'AS Hundal': 'Ajayveer Hundal',
    'Ansh Patel': 'Ansh Patel',
    'D Heyliger': 'Dilon Heyliger',
    'H Thaker': 'Harsh Thaker',
    'Jaskaran Singh': 'Jaskarandeep Singh',
    'Kaleem Sana': 'Kaleem Sana',
    'K Tathgur': 'Kanwarpal Tathgur',
    'NS Dhaliwal': 'Navneet Dhaliwal',
    'NR Kirton': 'Nicholas Kirton',
    'Ravinderpal Singh': 'Ravinderpal Singh',
    'Saad Bin Zafar': 'Saad Bin Zafar',
    'Shivam Sharma': 'Shivam Sharma',
    'S Movva': 'Shreyas Movva',
    'YS Samra': 'Yuvraj Samra',
    
    # UAE
    'Waseem Muhammad': 'Muhammad Waseem',
    'A Sharafu': 'Alishan Sharafu',
    'A Sharma': 'Aryansh Sharma',
    'D Parashar': 'Dhruv Parashar',
    'Haider Ali': 'Haider Ali',
    'H Kaushik': 'Harshit Kaushik',
    'Junaid Siddique': 'Junaid Siddique',
    'MR Kumar': 'Mayank Kumar',
    'Muhammad Arfan': 'Muhammad Arfan',
    'Muhammad Farooq': 'Muhammad Farooq',
    'Muhammad Jawadullah': 'Muhammad Jawadullah',
    'Muhammad Zohaib': 'Muhammad Zohaib',
    'Muhammad Rohid': 'Rohid Khan',
    'Sohaib Khan': 'Sohaib Khan',
    'Simranjeet Singh': 'Simranjeet Singh',
    
    # England
    'JC Buttler': 'Jos Buttler',
    'BM Duckett': 'Ben Duckett',
    'LA Dawson': 'Liam Dawson',
    'JG Bethell': 'Jacob Bethell',
    'PD Salt': 'Phil Salt',
    'WG Jacks': 'Will Jacks',
    'AU Rashid': 'Adil Rashid',
    'SM Curran': 'Sam Curran',
    'J Overton': 'Jamie Overton',
    'HC Brook': 'Harry Brook',
    'CJ Jordan': 'Chris Jordan',
    'Rehan Ahmed': 'Rehan Ahmed',
    'L Wood': 'Luke Wood',
    'JC Archer': 'Jofra Archer',
    
    # West Indies
    'SO Hetmyer': 'Shimron Hetmyer',
    'RL Chase': 'Roston Chase',
    'G Motie': 'Gudakesh Motie',
    'SD Hope': 'Shai Hope',
    'R Shepherd': 'Romario Shepherd',
    'R Powell': 'Rovman Powell',
    'SE Rutherford': 'Sherfane Rutherford',
    'BA King': 'Brandon King',
    'JO Holder': 'Jason Holder',
    'AJ Hosein': 'Akeal Hosein',
    'MW Forde': 'Matthew Forde',
    'J Blades': 'Jaden Blades',
    'S Joseph': 'Shamar Joseph',
    'Jayden Seales':'Jayden Seales',
    'J Charles': 'Johnson Charles',
    
    # Scotland
    'C Tear': 'Charlie Tear',
    'GT Main': 'George Main',
    'SM Sharif': 'Safyaan Sharif',
    'J Jarvis': 'Jack Jarvis',
    'CB Sole': 'Chris Sole',
    'BJ McMullen': 'Brandon McMullen',
    'CM McBride': 'Charlie McBride',
    'MA Jones': 'Michael Jones',
    'MA Leask': 'Michael Leask',
    'JJ Davidson': 'Jack Davidson',
    'CN Greaves': 'Chris Greaves',
    'MH Cross': 'Matthew Cross',
    'J Dickinson': 'Jack Dickinson',
    'LR Naylor': 'Liam Naylor',
    'OJ Hairs': 'Oliver Hairs',
    'FDW McCreath': 'Freddie McCreath',
    'HG Munsey': 'George Munsey',
    'MW Jones': 'Michael Jones',
    'BJ Currie': 'Brad Currie',
    'C Cassell': 'Charlie Cassell',
    'Hamza Tahir': 'Hamza Tahir',
    'MRJ Watt': 'Mark Watt',
    'RD Berrington': 'Richie Berrington',
    'BTJ Wheal': 'Brad Wheal',

    #Italy
     "AJ Mosca": "Anthony Mosca",
    "BAD Manenti": "Benjamin Manenti",
    "CJPF Kalugamage": "Crishan Jorge Kalugamage",
    "G Stewart": "Grant Stewart",
    "GP Meade": "Gian Piero Meade",
    "HJ Manenti": "Harry Manenti",
    "J Mosca": "Justin Mosca",
    "JT Smuts": "JJ Smuts",
    "Jaspreet Singh": "Jaspreet Singh",
    "M Campopiano": "Marcus Campopiano",
    "Syed Naqvi": "Syed Naqvi",
    "TJ Draca": "Thomas Draca",
    "WL Madsen": "Wayne Madsen",
    "Zain Ali": "Zain Ali",

    # Nepal
    "Aarif Sheikh": "Aarif Sheikh",
    "Aasif Sheikh": "Aasif Sheikh",
    "Basir Ahamad": "Basir Ahamad",
    "DS Airee": "Dipendra Singh Airee",
    "Gulsan Jha": "Gulshan Jha",
    "K Bhurtel": "Kushal Bhurtel",
    "Karan KC": "Karan KC",
    "LN Rajbanshi": "Lalit Rajbanshi",
    "Lokesh Bam": "Lokesh Bam",
    "NK Yadav": "Nandan Yadav",
    "RK Paudel": "Rohit Paudel",
    "S Jora": "Sundeep Jora",
    "S Lamichhane": "Sandeep Lamichhane",
    "Sompal Kami": "Sompal Kami"
}

In [48]:
batting_stats[batting_stats['batting_team']=='Nepal']['batter'].unique()

array(['A Bohara', 'AK Sah', 'Aakash Chand', 'Aarif Sheikh',
       'Aasif Sheikh', 'B Bhandari', 'B Sharki', 'B Yadav',
       'Basir Ahamad', 'D Khanal', 'DS Airee', 'Gulsan Jha', 'K Bhurtel',
       'K Thagunna', 'Karan KC', 'Kushal Malla', 'LN Rajbanshi',
       'Lokesh Bam', 'Mohammad Aadil Alam', 'NK Yadav', 'Pratis GC',
       'R Dhakal', 'RK Paudel', 'Rashid Khan', 'Rupesh Singh', 'S Dhakal',
       'S Jora', 'S Lamichhane', 'Shahab Alam', 'Sompal Kami'],
      dtype=object)

In [49]:
batting_stats['batter'] = batting_stats['batter'].map(name_mapping)

In [50]:
bowling_stats['bowler'] = bowling_stats['bowler'].map(name_mapping)

In [51]:
bowling_stats['bowler'].isnull().sum()

np.int64(227)

In [52]:
batting_stats= batting_stats.dropna()

In [53]:
bowling_stats = bowling_stats.dropna()

In [54]:
batting_stats[batting_stats['batting_team']=='India']

,batter,batting_team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,usual_batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
145,Axar Patel,India,267,313,21,13,81,19,12,117.228464,24.076923,0.303371,0.116105,8,122.105263,117.272727,109.677419,Finisher
193,Abhishek Sharma,India,691,1297,37,33,215,123,88,187.698987,39.303030,0.311143,0.305355,1,204.761905,196.913580,184.055118,Opener
275,Arshdeep Singh,India,48,46,13,6,28,5,2,95.833333,7.666667,0.583333,0.145833,9,100.000000,33.333333,0.000000,Tail
461,Varun Chakaravarthy,India,8,2,5,2,6,0,0,25.000000,1.000000,0.750000,0.000000,10,25.000000,0.000000,0.000000,Tail
752,Hardik Pandya,India,464,691,28,19,144,56,40,148.922414,36.368421,0.310345,0.206897,5,171.555556,124.553571,173.333333,Middle order
847,Ishan Kishan,India,99,215,4,4,26,22,16,217.171717,53.750000,0.262626,0.383838,2,271.428571,248.387097,195.081967,Opener
939,Jasprit Bumrah,India,4,4,3,3,3,1,0,100.000000,1.333333,0.750000,0.250000,10,100.000000,0.000000,0.000000,Tail
1129,Kuldeep Yadav,India,12,2,3,2,10,0,0,16.666667,1.000000,0.833333,0.000000,9,16.666667,0.000000,0.000000,Tail
1390,Mohammed Siraj,India,8,7,2,2,4,0,0,87.500000,3.500000,0.500000,0.000000,10,87.500000,0.000000,0.000000,Tail
1716,Rinku Singh,India,208,285,17,11,69,23,16,137.019231,25.909091,0.331731,0.187500,7,161.616162,106.153846,127.272727,Finisher


In [55]:
bowling_stats[bowling_stats['bowling_team']=='India']

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy
101,Axar Patel,India,545,686,30,215,37,90.833333,7.552294,0.394495,18.540541,14.729730,6.400000,7.216667,9.436364
138,Abhishek Sharma,India,155,225,17,43,6,25.833333,8.709677,0.277419,37.500000,25.833333,10.666667,7.834711,16.285714
201,Arshdeep Singh,India,694,942,30,279,55,115.666667,8.144092,0.402017,17.127273,12.618182,8.610879,5.731343,8.273196
334,Varun Chakaravarthy,India,649,807,28,265,59,108.166667,7.460709,0.408320,13.677966,11.000000,6.835443,7.747178,6.850394
537,Hardik Pandya,India,623,853,34,235,35,103.833333,8.215088,0.377207,24.371429,17.800000,8.416667,8.608696,7.734940
680,Jasprit Bumrah,India,503,579,22,232,33,83.833333,6.906561,0.461233,17.545455,15.242424,7.472393,6.880000,6.566038
819,Kuldeep Yadav,India,357,430,16,133,33,59.500000,7.226891,0.372549,13.030303,10.818182,5.454545,7.426710,10.000000
1002,Mohammed Siraj,India,123,119,6,62,3,20.500000,5.804878,0.504065,39.666667,41.000000,6.600000,4.666667,5.858824
1225,Rinku Singh,India,10,10,2,6,3,1.666667,6.000000,0.600000,3.333333,3.333333,6.000000,0.000000,0.000000
1288,Shivam Dube,India,254,348,18,78,18,42.333333,8.220472,0.307087,19.333333,14.111111,9.846154,8.039409,6.000000


In [56]:
bowling_roles = {
    # India
    'Axar Patel': 'Spin',
    'Abhishek Sharma': 'Spin',
    'Arshdeep Singh': 'Fast Medium',
    'Varun Chakaravarthy': 'Spin',
    'Hardik Pandya': 'Fast Medium',
    'Mohammed Siraj' : 'Fast',
    'Jasprit Bumrah': 'Fast',
    'Kuldeep Yadav': 'Spin',
    'Rinku Singh': 'Part-time',
    'Shivam Dube': 'Medium Pace',
    'Suryakumar Yadav': 'Part-time',
    'Tilak Varma': 'Spin',
    'Washington Sundar': 'Spin',
    'Sanju Samson': 'Part-time',
    'Ishan Kishan': 'Part-time',
    
    # Australia
    'Mitchell Marsh': 'Fast Medium',
    'Xavier Bartlett': 'Fast Medium',
    'Cooper Connolly': 'Spin',
    'Tim David': 'Part-time',
    'Ben Dwarshuis': 'Fast Medium',
    'Cameron Green': 'Fast Medium',
    'Nathan Ellis': 'Fast Medium',
    'Josh Hazlewood': 'Fast',
    'Travis Head': 'Spin',
    'Josh Inglis': 'Part-time',
    'Matthew Kuhnemann': 'Spin',
    'Glenn Maxwell': 'Spin',
    'Matthew Renshaw': 'Spin',
    'Marcus Stoinis': 'Fast Medium',
    'Adam Zampa': 'Spin',
    
    # Pakistan
    'Salman Ali Agha': 'Spin',
    'Abrar Ahmed': 'Spin',
    'Babar Azam': 'Part-time',
    'Faheem Ashraf': 'Fast Medium',
    'Fakhar Zaman': 'Part-time',
    'Naseem Shah': 'Fast',
    'Saim Ayub': 'Spin',
    'Shaheen Shah Afridi': 'Fast',
    'Shadab Khan': 'Spin',
    'Haris Rauf': 'Fast',
    'Mohammad Nawaz': 'Spin',
    'Mohammad Amir': 'Fast',
    'Iftikhar Ahmed': 'Spin',
    'Imad Wasim': 'Spin',
    
    # England
    'Jos Buttler': 'Part-time',
    'Ben Duckett': 'Part-time',
    'Liam Dawson': 'Spin',
    'Jordan Cox': 'Part-time',
    'Brydon Carse': 'Fast',
    'Jamie Smith': 'Part-time',
    'Reece Topley': 'Fast Medium',
    'Saqib Mahmood': 'Fast Medium',
    'Jofra Archer': 'Fast',
    'Mark Wood': 'Fast',
    'Luke Wood': 'Fast Medium',
    'Jacob Bethell': 'Spin',
    'Tom Banton': 'Part-time',
    'Moeen Ali': 'Spin',
    'Dan Mousley': 'Spin',
    'Phil Salt': 'Part-time',
    'Jonny Bairstow': 'Part-time',
    'Gus Atkinson': 'Fast',
    'Will Jacks': 'Spin',
    'Adil Rashid': 'Spin',
    'Sam Curran': 'Fast Medium',
    'Jamie Overton': 'Fast',
    'Harry Brook': 'Part-time',
    'Chris Jordan': 'Fast Medium',
    'Rehan Ahmed': 'Spin',
    'Liam Livingstone': 'Spin',
    
    # South Africa
    'Aiden Markram': 'Spin',
    'Corbin Bosch': 'Fast Medium',
    'Dewald Brevis': 'Spin',
    'Quinton de Kock': 'Part-time',
    'Marco Jansen': 'Fast Medium',
    'George Linde': 'Spin',
    'Keshav Maharaj': 'Spin',
    'Kwena Maphaka': 'Fast',
    'David Miller': 'Part-time',
    'Lungi Ngidi': 'Fast',
    'Anrich Nortje': 'Fast',
    'Kagiso Rabada': 'Fast',
    'Ryan Rickelton': 'Part-time',
    'Jason Smith': 'Part-time',
    'Tristan Stubbs': 'Part-time',
    
    # New Zealand
    'Mitchell Santner': 'Spin',
    'Finn Allen': 'Part-time',
    'Michael Bracewell': 'Spin',
    'Mark Chapman': 'Spin',
    'Devon Conway': 'Part-time',
    'Jacob Duffy': 'Fast Medium',
    'Lockie Ferguson': 'Fast',
    'Matt Henry': 'Fast Medium',
    'Kyle Jamieson': 'Fast',
    'Daryl Mitchell': 'Medium Pace',
    'James Neesham': 'Fast Medium',
    'Glenn Phillips': 'Spin',
    'Rachin Ravindra': 'Spin',
    'Tim Seifert': 'Part-time',
    'Ish Sodhi': 'Spin',
    
    # West Indies
    'Shimron Hetmyer': 'Part-time',
    'Roston Chase': 'Spin',
    'Gudakesh Motie': 'Spin',
    'Fabian Allen': 'Spin',
    'Shamar Springer': 'Fast Medium',
    'Shai Hope': 'Part-time',
    'Romario Shepherd': 'Fast Medium',
    'Alzarri Joseph': 'Fast',
    'Rovman Powell': 'Medium Pace',
    'Ackeem Auguste': 'Part-time',
    'Obed McCoy': 'Fast Medium',
    'Sherfane Rutherford': 'Medium Pace',
    'Brandon King': 'Part-time',
    'Evin Lewis': 'Part-time',
    'Amir Jangoo': 'Part-time',
    'Alick Athanaze': 'Spin',
    'Nicholas Bidaisee': 'Fast Medium',
    'Keacy Carty': 'Part-time',
    'Justin Greaves': 'Medium Pace',
    'Jason Holder': 'Fast Medium',
    'Akeal Hosein': 'Spin',
    'Jayden Seales': 'Fast',
    'Khary Pierre': 'Spin',
    'Matthew Forde': 'Fast Medium',
    'Johnson Charles': 'Part-time',
    'Andre Russell': 'Fast Medium',
    'Shamar Joseph': 'Fast',
    'Andre Fletcher': 'Part-time',
    'Nicholas Pooran': 'Part-time',
    'Kyle Mayers': 'Fast Medium',
    
    # Sri Lanka
    'Dasun Shanaka': 'Fast Medium',
    'Pathum Nissanka': 'Part-time',
    'Kamil Mishara': 'Part-time',
    'Kusal Mendis': 'Part-time',
    'Kamindu Mendis': 'Spin',
    'Kusal Janith Perera': 'Part-time',
    'Charith Asalanka': 'Spin',
    'Janith Liyanage': 'Medium Pace',
    'Pavan Rathnayake': 'Fast Medium',
    'Wanindu Hasaranga': 'Spin',
    'Dunith Wellalage': 'Spin',
    'Maheesh Theekshana': 'Spin',
    'Dushmantha Chameera': 'Fast',
    'Matheesha Pathirana': 'Fast',
    'Eshan Malinga': 'Fast',
    
    # Add more teams as needed...
    # Zimbabwe
    'Sikandar Raza': 'Spin',
    'Brian Bennett': 'Spin',
    'Ryan Burl': 'Spin',
    'Graeme Cremer': 'Spin',
    'Bradley Evans': 'Fast Medium',
    'Clive Madande': 'Part-time',
    'Tinotenda Maposa': 'Fast Medium',
    'Tadiwanashe Marumani': 'Part-time',
    'Wellington Masakadza': 'Spin',
    'Tony Munyonga': 'Fast Medium',
    'Tashinga Musekiwa': 'Spin',
    'Blessing Muzarabani': 'Fast',
    'Dion Myers': 'Spin',
    'Richard Ngarava': 'Fast Medium',
    'Brendan Taylor': 'Part-time',
    
    # Ireland
    'Paul Stirling': 'Spin',
    'Mark Adair': 'Fast Medium',
    'Ross Adair': 'Part-time',
    'Ben Calitz': 'Part-time',
    'Curtis Campher': 'Fast Medium',
    'Gareth Delany': 'Spin',
    'George Dockrell': 'Spin',
    'Matthew Humphreys': 'Spin',
    'Josh Little': 'Fast Medium',
    'Barry McCarthy': 'Fast Medium',
    'Harry Tector': 'Part-time',
    'Tim Tector': 'Part-time',
    'Lorcan Tucker': 'Part-time',
    'Ben White': 'Spin',
    'Craig Young': 'Fast Medium',
    
    # Scotland
    'Charlie Tear': 'Fast Medium',
    'George Main': 'Fast Medium',
    'Safyaan Sharif': 'Fast Medium',
    'Jack Jarvis': 'Fast Medium',
    'Chris Sole': 'Fast Medium',
    'Brandon McMullen': 'Fast Medium',
    'Charlie McBride': 'Part-time',
    'Michael Jones': 'Part-time',
    'Michael Leask': 'Spin',
    'Jack Davidson': 'Fast Medium',
    'Chris Greaves': 'Spin',
    'Matthew Cross': 'Part-time',
    'George Munsey': 'Part-time',
    'Brad Currie': 'Fast Medium',
    'Charlie Cassell': 'Fast Medium',
    'Hamza Tahir': 'Spin',
    'Mark Watt': 'Spin',
    'Richie Berrington': 'Medium Pace',
    'Brad Wheal': 'Fast',
    
    # Netherlands
    'Scott Edwards': 'Part-time',
    'Colin Ackermann': 'Spin',
    'Noah Croes': 'Part-time',
    'Bas de Leede': 'Fast Medium',
    'Aryan Dutt': 'Spin',
    'Fred Klaassen': 'Fast Medium',
    'Kyle Klein': 'Fast Medium',
    'Michael Levitt': 'Part-time',
    'Zach Lion-Cachet': 'Part-time',
    "Max O'Dowd": 'Part-time',
    'Logan van Beek': 'Fast Medium',
    'Timm van der Gugten': 'Fast Medium',
    'Roelof van der Merwe': 'Spin',
    'Paul van Meekeren': 'Fast',
    'Saqib Zulfiqar': 'Spin',
    
    # Namibia
    'Gerhard Erasmus': 'Spin',
    'Zane Green': 'Part-time',
    'Bernard Scholtz': 'Spin',
    'Ruben Trumpelmann': 'Fast Medium',
    'JJ Smit': 'Fast Medium',
    'Jan Frylinck': 'Fast Medium',
    'Louren Steenkamp': 'Part-time',
    'Malan Kruger': 'Part-time',
    'Nicol Loftie-Eaton': 'Spin',
    'Jack Brassell': 'Fast Medium',
    'Ben Shikongo': 'Fast',
    'JC Balt': 'Part-time',
    'Dylan Leicher': 'Part-time',
    
    # Oman
    'Jatinder Singh': 'Part-time',
    'Vinayak Shukla': 'Part-time',
    'Mohammad Nadeem': 'Fast Medium',
    'Shakeel Ahmad': 'Medium Pace',
    'Hammad Mirza': 'Part-time',
    'Wasim Ali': 'Part-time',
    'Karan Sonavale': 'Part-time',
    'Shah Faisal': 'Fast Medium',
    'Nadeem Khan': 'Part-time',
    'Sufyan Mehmood': 'Fast Medium',
    'Jay Odedra': 'Spin',
    'Shafiq Jan': 'Part-time',
    'Ashish Odedara': 'Spin',
    'Jiten Ramanandi': 'Part-time',
    'Aamir Kaleem': 'Fast Medium',
    
    # USA
    'Monank Patel': 'Part-time',
    'Jessy Singh': 'Fast Medium',
    'Andries Gous': 'Part-time',
    'Shehan Jayasuriya': 'Spin',
    'Milind Kumar': 'Part-time',
    'Shayan Jahangir': 'Part-time',
    'Saiteja Mukkamala': 'Part-time',
    'Sanjay Krishnamurthi': 'Part-time',
    'Harmeet Singh': 'Spin',
    'Nosthush Kenjige': 'Spin',
    'Shadley Van Schalkwyk': 'Fast Medium',
    'Saurabh Netravalkar': 'Fast Medium',
    'Ali Khan': 'Fast',
    
    # Canada
    'Dilpreet Bajwa': 'Fast Medium',
    'Ajayveer Hundal': 'Part-time',
    'Ansh Patel': 'Part-time',
    'Dilon Heyliger': 'Fast Medium',
    'Harsh Thaker': 'Spin',
    'Jaskarandeep Singh': 'Fast Medium',
    'Kaleem Sana': 'Fast Medium',
    'Kanwarpal Tathgur': 'Part-time',
    'Navneet Dhaliwal': 'Spin',
    'Nicholas Kirton': 'Part-time',
    'Ravinderpal Singh': 'Fast Medium',
    'Saad Bin Zafar': 'Spin',
    'Shivam Sharma': 'Part-time',
    'Shreyas Movva': 'Part-time',
    'Yuvraj Samra': 'Part-time',
    
    # UAE
    'Muhammad Waseem': 'Part-time',
    'Alishan Sharafu': 'Spin',
    'Aryansh Sharma': 'Part-time',
    'Dhruv Parashar': 'Part-time',
    'Haider Ali': 'Part-time',
    'Harshit Kaushik': 'Part-time',
    'Junaid Siddique': 'Fast Medium',
    'Mayank Kumar': 'Part-time',
    'Muhammad Arfan': 'Fast Medium',
    'Muhammad Farooq': 'Fast Medium',
    'Muhammad Jawadullah': 'Fast',
    'Muhammad Zohaib': 'Part-time',
    'Rohid Khan': 'Part-time',
    'Sohaib Khan': 'Part-time',
    'Simranjeet Singh': 'Fast Medium',

     # Italy

    "Anthony Mosca": "Fast Medium",
    "Benjamin Manenti": "Spin",
    "Crishan Jorge Kalugamage": "Part-time",
    "Grant Stewart": "Fast Medium",
    "Gian Piero Meade": "Part-time",
    "Harry Manenti": "Spin",
    "Justin Mosca": "Part-time",
    "JJ Smuts": "Spin",
    "Jaspreet Singh": "Fast",
    "Marcus Campopiano": "Part-time",
    "Syed Naqvi": "Spin",
    "Thomas Draca": "Fast Medium",
    "Wayne Madsen": "Part-time",
    "Zain Ali": "Fast Medium",

    # Nepal

    "Aarif Sheikh": "Part-time",
    "Aasif Sheikh": "Part-time",
    "Basir Ahamad": "Spin",
    "Dipendra Singh Airee": "Spin",
    "Gulshan Jha": "Fast Medium",
    "Kushal Bhurtel": "Part-time",
    "Karan KC": "Fast",
    "Lalit Rajbanshi": "Spin",
    "Lokesh Bam": "Part-time",
    "Nandan Yadav": "Fast Medium",
    "Rohit Paudel": "Part-time",
    "Sundeep Jora": "Part-time",
    "Sandeep Lamichhane": "Spin",
    "Sompal Kami": "Fast Medium"
}

In [57]:
bowling_stats['bowling_role'] = bowling_stats['bowler'].map(bowling_roles)

In [58]:
bowling_stats.head()

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role
16,Aryan Dutt,Netherlands,449,501,21,187,22,74.833333,6.694878,0.416481,22.772727,20.409091,7.687500,6.940299,6.310757,Spin
39,Anrich Nortje,South Africa,303,389,13,120,14,50.500000,7.702970,0.396040,27.785714,21.642857,8.089888,6.228261,15.600000,Fast
49,Alishan Sharafu,United Arab Emirates,1,3,1,0,0,0.166667,18.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,Spin
60,Adam Zampa,Australia,776,1050,34,253,55,129.333333,8.118557,0.326031,19.090909,14.109091,8.712329,7.941748,10.000000,Spin
80,Graeme Cremer,Zimbabwe,42,46,2,18,2,7.000000,6.571429,0.428571,23.000000,21.000000,1.500000,8.600000,0.000000,Spin


In [59]:
bowling_stats[bowling_stats['bowling_role'].isnull()]

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role
624,Jaden Blades,West Indies,131,181,6,53,6,21.833333,8.290076,0.404580,30.166667,21.833333,8.756757,8.000000,8.192308,NaN
632,Jack Dickinson,Scotland,24,23,1,6,1,4.000000,5.750000,0.250000,23.000000,24.000000,7.000000,5.333333,0.000000,NaN
1397,Mohammad Salman Mirza,Pakistan,303,316,13,145,20,50.500000,6.257426,0.478548,15.800000,15.150000,7.255814,4.285714,5.914286,NaN
1543,Usman Tariq,Pakistan,66,60,3,26,8,11.000000,5.454545,0.393939,7.500000,8.250000,6.000000,5.357143,0.000000,NaN


In [60]:
player_roles = {
    'Jaden Blades': 'Fast Medium',
    'Jack Dickinson': 'Part-time',
    'Mohammad Salman Mirza': 'Spin',
    'Usman Tariq': 'Spin'
}

for player, role in player_roles.items():
    bowling_stats.loc[bowling_stats['bowler'] == player, 'bowling_role'] = role

In [61]:
bowling_stats[bowling_stats['bowling_role'].isnull()]

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role


In [62]:
batting_stats.head()

,batter,batting_team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,usual_batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
19,Aryan Dutt,Netherlands,114,114,13,7,39,7,3,100.000000,16.285714,0.342105,0.087719,9,113.698630,75.609756,0.000000,Tail
61,Anrich Nortje,South Africa,17,14,3,2,7,0,1,82.352941,7.000000,0.411765,0.058824,9,100.000000,25.000000,0.000000,Tail
76,Alishan Sharafu,United Arab Emirates,1027,1386,44,38,382,135,61,134.956183,36.473684,0.371957,0.190847,1,148.275862,128.260870,138.022814,Opener
77,Aryansh Sharma,United Arab Emirates,134,165,7,7,56,16,6,123.134328,23.571429,0.417910,0.164179,1,175.000000,107.407407,125.242718,Opener
89,Adam Zampa,Australia,27,19,10,5,12,2,0,70.370370,3.800000,0.444444,0.074074,10,69.230769,100.000000,0.000000,Tail


In [63]:
bowling_stats.head()

,bowler,bowling_team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role
16,Aryan Dutt,Netherlands,449,501,21,187,22,74.833333,6.694878,0.416481,22.772727,20.409091,7.687500,6.940299,6.310757,Spin
39,Anrich Nortje,South Africa,303,389,13,120,14,50.500000,7.702970,0.396040,27.785714,21.642857,8.089888,6.228261,15.600000,Fast
49,Alishan Sharafu,United Arab Emirates,1,3,1,0,0,0.166667,18.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,Spin
60,Adam Zampa,Australia,776,1050,34,253,55,129.333333,8.118557,0.326031,19.090909,14.109091,8.712329,7.941748,10.000000,Spin
80,Graeme Cremer,Zimbabwe,42,46,2,18,2,7.000000,6.571429,0.428571,23.000000,21.000000,1.500000,8.600000,0.000000,Spin


In [64]:
batting_stats['batting_team'].nunique()

19

In [65]:
bowling_stats['bowling_team'].nunique()

19

In [66]:
batting_stats.columns = ['player', 'team', 'balls_faced', 'runs', 'matches_played',
       'dismissals', 'dot_balls', 'fours', 'sixes', 'strike_rate',
       'batting_average', 'dot_ball_pct', 'boundary_pct',
       'batting_position', 'death_strike_rate', 'middle_strike_rate',
       'pp_strike_rate', 'batting_role']

In [67]:
bowling_stats.columns = ['player', 'team', 'balls_bowled', 'runs_conceded',
       'matches_played', 'dot_balls', 'wickets', 'overs_bowled', 'economy',
       'dot_ball_pct', 'bowling_average', 'bowling_strike_rate',
       'death_economy', 'middle_economy', 'pp_economy', 'bowling_role']

In [68]:
batting_stats.columns

Index(['player', 'team', 'balls_faced', 'runs', 'matches_played', 'dismissals',
       'dot_balls', 'fours', 'sixes', 'strike_rate', 'batting_average',
       'dot_ball_pct', 'boundary_pct', 'batting_position', 'death_strike_rate',
       'middle_strike_rate', 'pp_strike_rate', 'batting_role'],
      dtype='object')

In [69]:
bowling_stats.columns

Index(['player', 'team', 'balls_bowled', 'runs_conceded', 'matches_played',
       'dot_balls', 'wickets', 'overs_bowled', 'economy', 'dot_ball_pct',
       'bowling_average', 'bowling_strike_rate', 'death_economy',
       'middle_economy', 'pp_economy', 'bowling_role'],
      dtype='object')

In [366]:
player_data = pd.merge(left=batting_stats,right=bowling_stats,how='outer',on=['player','team'])

In [367]:
len(player_data)

281

In [368]:
player_data.columns

Index(['player', 'team', 'balls_faced', 'runs', 'matches_played_x',
       'dismissals', 'dot_balls_x', 'fours', 'sixes', 'strike_rate',
       'batting_average', 'dot_ball_pct_x', 'boundary_pct', 'batting_position',
       'death_strike_rate', 'middle_strike_rate', 'pp_strike_rate',
       'batting_role', 'balls_bowled', 'runs_conceded', 'matches_played_y',
       'dot_balls_y', 'wickets', 'overs_bowled', 'economy', 'dot_ball_pct_y',
       'bowling_average', 'bowling_strike_rate', 'death_economy',
       'middle_economy', 'pp_economy', 'bowling_role'],
      dtype='object')

### Assigning Player Roles

In [369]:
player_roles = {
    # Australia
    'Marcus Stoinis': 'Allrounder', 'Travis Head': 'Batter', 'Josh Hazlewood': 'Bowler', 
    'Ben Dwarshuis': 'Bowler', 'Matthew Renshaw': 'Batter', 'Mitchell Marsh': 'Allrounder', 
    'Tim David': 'Batter', 'Cameron Green': 'Allrounder', 'Nathan Ellis': 'Bowler', 
    'Matthew Kuhnemann': 'Bowler', 'Josh Inglis': 'WicketKeeper', 'Xavier Bartlett': 'Bowler', 
    'Cooper Connolly': 'Allrounder', 'Adam Zampa': 'Bowler', 'Glenn Maxwell': 'Allrounder',
    
    # Canada
    'Shreyas Movva': 'WicketKeeper', 'Navneet Dhaliwal': 'Batter', 'Nicholas Kirton': 'Batter', 
    'Harsh Thaker': 'Allrounder', 'Ravinderpal Singh': 'Batter', 'Dilon Heyliger': 'Allrounder', 
    'Dilpreet Bajwa': 'Batter', 'Shivam Sharma': 'Bowler', 'Jaskarandeep Singh': 'Bowler', 
    'Saad Bin Zafar': 'Allrounder', 'Ajayveer Hundal': 'Batter', 'Yuvraj Samra': 'Batter', 
    'Ansh Patel': 'Bowler', 'Kanwarpal Tathgur': 'Batter', 'Kaleem Sana': 'Bowler',
    
    # England
    'Adil Rashid': 'Bowler', 'Harry Brook': 'Batter', 'Phil Salt': 'WicketKeeper', 
    'Jacob Bethell': 'Allrounder', 'Jamie Overton': 'Allrounder', 'Chris Jordan': 'Bowler', 
    'Will Jacks': 'Allrounder', 'Rehan Ahmed': 'Bowler', 'Sam Curran': 'Allrounder', 
    'Jofra Archer': 'Bowler', 'Luke Wood': 'Bowler', 'Jos Buttler': 'WicketKeeper', 
    'Liam Dawson': 'Allrounder', 'Ben Duckett': 'Batter',
    
    # India
    'Sanju Samson': 'WicketKeeper', 'Hardik Pandya': 'Allrounder', 'Abhishek Sharma': 'Allrounder', 
    'Kuldeep Yadav': 'Bowler', 'Axar Patel': 'Allrounder', 'Arshdeep Singh': 'Bowler', 
    'Varun Chakaravarthy': 'Bowler', 'Jasprit Bumrah': 'Bowler', 'Mohammed Siraj': 'Bowler', 
    'Shivam Dube': 'Allrounder', 'Tilak Varma': 'Batter', 'Rinku Singh': 'Batter', 
    'Washington Sundar': 'Allrounder', 'Ishan Kishan': 'WicketKeeper', 'Suryakumar Yadav': 'Batter',
    
    # Ireland
    'Paul Stirling': 'Batter', 'George Dockrell': 'Allrounder', 'Harry Tector': 'Batter', 
    'Matthew Humphreys': 'Bowler', 'Mark Adair': 'Bowler', 'Gareth Delany': 'Allrounder', 
    'Ross Adair': 'Batter', 'Josh Little': 'Bowler', 'Lorcan Tucker': 'WicketKeeper', 
    'Tim Tector': 'Batter', 'Craig Young': 'Bowler', 'Ben White': 'Bowler', 
    'Barry McCarthy': 'Bowler', 'Ben Calitz': 'Batter', 'Curtis Campher': 'Allrounder',
    
    # Italy / Others
    'Thomas Draca': 'Bowler', 'Syed Naqvi': 'Allrounder', 'Jaspreet Singh': 'Bowler', 
    'Grant Stewart': 'Allrounder', 'Benjamin Manenti': 'Allrounder', 'Harry Manenti': 'Allrounder', 
    'Justin Mosca': 'Batter', 'Anthony Mosca': 'Batter', 'Wayne Madsen': 'Batter', 
    'Marcus Campopiano': 'Batter', 'Zain Ali': 'Bowler', 'JJ Smuts': 'Allrounder', 
    'Gian Piero Meade': 'Allrounder', 'Crishan Jorge Kalugamage': 'Allrounder',
    
    # Namibia
    'Jack Brassell': 'Bowler', 'Bernard Scholtz': 'Bowler', 'Jan Frylinck': 'Allrounder', 
    'JJ Smit': 'Allrounder', 'Ben Shikongo': 'Bowler', 'Malan Kruger': 'Batter', 
    'Louren Steenkamp': 'Batter', 'Dylan Leicher': 'Allrounder', 'JC Balt': 'Batter', 
    'Ruben Trumpelmann': 'Bowler', 'Zane Green': 'WicketKeeper', 'Nicol Loftie-Eaton': 'Allrounder', 
    'Gerhard Erasmus': 'Allrounder',
    
    # Nepal
    'Aarif Sheikh': 'Allrounder', 'Basir Ahamad': 'Allrounder', 'Lokesh Bam': 'Batter', 
    'Karan KC': 'Bowler', 'Sandeep Lamichhane': 'Bowler', 'Gulshan Jha': 'Allrounder', 
    'Dipendra Singh Airee': 'Allrounder', 'Sompal Kami': 'Allrounder', 'Rohit Paudel': 'Batter', 
    'Aasif Sheikh': 'WicketKeeper', 'Nandan Yadav': 'Bowler', 'Sundeep Jora': 'Batter', 
    'Lalit Rajbanshi': 'Bowler', 'Kushal Bhurtel': 'Batter',
    
    # Netherlands
    'Zach Lion-Cachet': 'Batter', 'Kyle Klein': 'Bowler', 'Logan van Beek': 'Bowler', 
    'Saqib Zulfiqar': 'Allrounder', 'Aryan Dutt': 'Bowler', 'Bas de Leede': 'Allrounder', 
    'Roelof van der Merwe': 'Allrounder', 'Scott Edwards': 'WicketKeeper', "Max O'Dowd": 'Batter', 
    'Michael Levitt': 'Batter', 'Colin Ackermann': 'Allrounder', 'Timm van der Gugten': 'Bowler', 
    'Noah Croes': 'WicketKeeper', 'Paul van Meekeren': 'Bowler',
    
    # New Zealand
    'Glenn Phillips': 'Batter', 'Lockie Ferguson': 'Bowler', 'Jacob Duffy': 'Bowler', 
    'Kyle Jamieson': 'Bowler', 'James Neesham': 'Allrounder', 'Finn Allen': 'Batter', 
    'Tim Seifert': 'WicketKeeper', 'Mitchell Santner': 'Allrounder', 'Daryl Mitchell': 'Allrounder', 
    'Mark Chapman': 'Batter', 'Matt Henry': 'Bowler', 'Rachin Ravindra': 'Allrounder', 
    'Devon Conway': 'WicketKeeper', 'Michael Bracewell': 'Allrounder', 'Ish Sodhi': 'Bowler',
    
    # Oman
    'Mohammad Nadeem': 'Allrounder', 'Shafiq Jan': 'Bowler', 'Nadeem Khan': 'Batter', 
    'Vinayak Shukla': 'WicketKeeper', 'Sufyan Mehmood': 'Bowler', 'Shah Faisal': 'Bowler', 
    'Shakeel Ahmad': 'Bowler', 'Wasim Ali': 'Allrounder', 'Aamir Kaleem': 'Allrounder', 
    'Hammad Mirza': 'Batter', 'Ashish Odedara': 'Batter', 'Jatinder Singh': 'Batter', 
    'Karan Sonavale': 'Allrounder', 'Jay Odedra': 'Bowler', 'Jiten Ramanandi': 'Bowler',
    
    # Pakistan
    'Faheem Ashraf': 'Allrounder', 'Shaheen Shah Afridi': 'Bowler', 'Shadab Khan': 'Allrounder', 
    'Salman Ali Agha': 'Allrounder', 'Saim Ayub': 'Batter', 'Sahibzada Farhan': 'Batter', 
    'Naseem Shah': 'Bowler', 'Mohammad Nawaz': 'Allrounder', 'Mohammad Salman Mirza': 'Bowler', 
    'Fakhar Zaman': 'Batter', 'Babar Azam': 'Batter', 'Usman Khan': 'Batter', 
    'Usman Tariq': 'Bowler', 'Khawaja Nafay': 'Batter', 'Abrar Ahmed': 'Bowler',
    
    # Scotland
    'Jack Davidson': 'Bowler', 'Jack Dickinson': 'Batter', 'Jack Jarvis': 'Allrounder', 
    'Safyaan Sharif': 'Bowler', 'Oliver Hairs': 'Batter', 'Freddie McCreath': 'Batter', 
    'Richie Berrington': 'Batter', 'Hamza Tahir': 'Bowler', 'Liam Naylor': 'Batter', 
    'Chris Sole': 'Bowler', 'George Main': 'Bowler', 'Mark Watt': 'Bowler', 
    'Matthew Cross': 'WicketKeeper', 'Brad Currie': 'Bowler', 'Brad Wheal': 'Bowler', 
    'Charlie Cassell': 'Bowler', 'Charlie McBride': 'Allrounder', 'Brandon McMullen': 'Allrounder', 
    'Michael Jones': 'Batter', 'Michael Leask': 'Allrounder', 'Chris Greaves': 'Allrounder', 
    'Charlie Tear': 'WicketKeeper', 'George Munsey': 'Batter',
    
    # South Africa
    'Corbin Bosch': 'Allrounder', 'Ryan Rickelton': 'WicketKeeper', 'Dewald Brevis': 'Batter', 
    'Tristan Stubbs': 'WicketKeeper', 'David Miller': 'Batter', 'Anrich Nortje': 'Bowler', 
    'George Linde': 'Allrounder', 'Aiden Markram': 'Allrounder', 'Kwena Maphaka': 'Bowler', 
    'Quinton de Kock': 'WicketKeeper', 'Lungi Ngidi': 'Bowler', 'Marco Jansen': 'Allrounder', 
    'Keshav Maharaj': 'Bowler', 'Kagiso Rabada': 'Bowler', 'Jason Smith': 'Allrounder',
    
    # Sri Lanka
    'Maheesh Theekshana': 'Bowler', 'Kusal Janith Perera': 'WicketKeeper', 'Kamindu Mendis': 'Allrounder', 
    'Matheesha Pathirana': 'Bowler', 'Kamil Mishara': 'WicketKeeper', 'Charith Asalanka': 'Batter', 
    'Janith Liyanage': 'Allrounder', 'Dasun Shanaka': 'Allrounder', 'Dunith Wellalage': 'Allrounder', 
    'Kusal Mendis': 'WicketKeeper', 'Pathum Nissanka': 'Batter', 'Eshan Malinga': 'Bowler', 
    'Dushmantha Chameera': 'Bowler', 'Pavan Rathnayake': 'Batter',
    
    # UAE
    'Junaid Siddique': 'Bowler', 'Haider Ali': 'Bowler', 'Alishan Sharafu': 'Batter', 
    'Rohid Khan': 'Bowler', 'Muhammad Arfan': 'Bowler', 'Aryansh Sharma': 'WicketKeeper', 
    'Dhruv Parashar': 'Allrounder', 'Mayank Kumar': 'Batter', 'Harshit Kaushik': 'Allrounder', 
    'Muhammad Jawadullah': 'Bowler', 'Muhammad Waseem': 'Batter', 'Muhammad Zohaib': 'Bowler', 
    'Muhammad Farooq': 'Allrounder',
    
    # USA
    'Jessy Singh': 'Bowler', 'Milind Kumar': 'Batter', 'Harmeet Singh': 'Allrounder', 
    'Nosthush Kenjige': 'Bowler', 'Shayan Jahangir': 'WicketKeeper', 'Shadley Van Schalkwyk': 'Allrounder', 
    'Monank Patel': 'WicketKeeper', 'Saurabh Netravalkar': 'Bowler', 'Sanjay Krishnamurthi': 'Allrounder', 
    'Saiteja Mukkamala': 'Batter', 'Andries Gous': 'WicketKeeper', 'Ali Khan': 'Bowler',
    
    # West Indies
    'Brandon King': 'Batter', 'Akeal Hosein': 'Bowler', 'Gudakesh Motie': 'Bowler', 
    'Romario Shepherd': 'Allrounder', 'Matthew Forde': 'Bowler', 'Jaden Blades': 'Bowler', 
    'Sherfane Rutherford': 'Batter', 'Jason Holder': 'Allrounder', 'Shai Hope': 'WicketKeeper', 
    'Shamar Joseph': 'Bowler', 'Shimron Hetmyer': 'Batter', 'Rovman Powell': 'Batter', 
    'Roston Chase': 'Allrounder', 'Johnson Charles': 'WicketKeeper',
    
    # Zimbabwe
    'Sikandar Raza': 'Allrounder', 'Tinotenda Maposa': 'Bowler', 'Blessing Muzarabani': 'Bowler', 
    'Tashinga Musekiwa': 'Allrounder', 'Graeme Cremer': 'Bowler', 'Brian Bennett': 'Allrounder', 
    'Clive Madande': 'WicketKeeper', 'Brendan Taylor': 'WicketKeeper', 'Wellington Masakadza': 'Bowler', 
    'Bradley Evans': 'Allrounder', 'Ryan Burl': 'Allrounder', 'Richard Ngarava': 'Bowler', 
    'Dion Myers': 'Batter', 'Tony Munyonga': 'Allrounder', 'Tadiwanashe Marumani': 'Batter'
}

In [370]:
player_data['player_role'] = player_data['player'].map(player_roles)

In [371]:
player_data.columns

Index(['player', 'team', 'balls_faced', 'runs', 'matches_played_x',
       'dismissals', 'dot_balls_x', 'fours', 'sixes', 'strike_rate',
       'batting_average', 'dot_ball_pct_x', 'boundary_pct', 'batting_position',
       'death_strike_rate', 'middle_strike_rate', 'pp_strike_rate',
       'batting_role', 'balls_bowled', 'runs_conceded', 'matches_played_y',
       'dot_balls_y', 'wickets', 'overs_bowled', 'economy', 'dot_ball_pct_y',
       'bowling_average', 'bowling_strike_rate', 'death_economy',
       'middle_economy', 'pp_economy', 'bowling_role', 'player_role'],
      dtype='object')

In [372]:
player_data = player_data[['player','team','player_role','batting_position','batting_role','bowling_role']]

In [373]:
player_data[player_data['team']=='England'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role
204,Phil Salt,England,WicketKeeper,1.0,Opener,NaN
26,Ben Duckett,England,Batter,2.0,Opener,NaN
275,Will Jacks,England,Allrounder,2.0,Opener,Spin
123,Jos Buttler,England,WicketKeeper,2.0,Opener,NaN
105,Jacob Bethell,England,Allrounder,3.0,Top order,Spin
208,Rehan Ahmed,England,Bowler,4.0,Top order,Spin
91,Harry Brook,England,Batter,4.0,Top order,NaN
228,Sam Curran,England,Allrounder,6.0,Middle order,Fast Medium
109,Jamie Overton,England,Allrounder,7.0,Finisher,Fast
146,Liam Dawson,England,Allrounder,8.0,Finisher,Spin


In [374]:
batting_stats[batting_stats['team']=='England'].sort_values(by='batting_position')

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
1594,Phil Salt,England,592,948,28,24,176,98,44,160.135135,39.500000,0.297297,0.239865,1,211.627907,143.975904,161.357702,Opener
366,Ben Duckett,England,132,212,9,9,40,29,6,160.606061,23.555556,0.303030,0.265152,2,0.000000,118.421053,177.659574,Opener
2214,Will Jacks,England,188,223,15,13,64,18,8,118.617021,17.153846,0.340426,0.138298,2,106.250000,89.552239,139.047619,Opener
927,Jos Buttler,England,657,1023,30,28,198,99,50,155.707763,36.535714,0.301370,0.226788,2,157.142857,148.529412,159.058824,Opener
936,Jacob Bethell,England,290,417,21,16,100,31,26,143.793103,26.062500,0.344828,0.196552,3,192.307692,137.037037,118.888889,Top order
1761,Rehan Ahmed,England,20,22,3,1,10,2,1,110.000000,22.000000,0.500000,0.150000,4,127.272727,88.888889,0.000000,Top order
747,Harry Brook,England,338,524,21,15,92,46,23,155.029586,34.933333,0.272189,0.204142,4,167.123288,160.975610,120.000000,Top order
1903,Sam Curran,England,191,269,13,7,46,16,12,140.837696,38.428571,0.240838,0.146597,6,165.333333,112.745098,214.285714,Middle order
897,Jamie Overton,England,55,56,10,7,20,4,1,101.818182,8.000000,0.363636,0.090909,7,106.250000,95.652174,0.000000,Finisher
1158,Liam Dawson,England,20,14,1,1,8,1,0,70.000000,14.000000,0.400000,0.050000,8,25.000000,100.000000,0.000000,Finisher


In [375]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin
231,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN
253,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time
258,Tilak Varma,India,Batter,4.0,Top order,Spin
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium
246,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace
211,Rinku Singh,India,Batter,7.0,Finisher,Part-time
271,Washington Sundar,India,Allrounder,7.0,Finisher,Spin
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin


In [376]:
matches_df.head()

,match_id,date,venue,city,gender,match_type,team_type,team_1,team_2,toss_winner,toss_decision,winner
0,1001349,2017-02-17,Melbourne Cricket Ground,None,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka
1,1001351,2017-02-19,"Simonds Stadium, South Geelong",Victoria,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka
2,1001353,2017-02-22,Adelaide Oval,None,male,T20,international,Australia,Sri Lanka,Sri Lanka,field,Australia
3,1004729,2016-09-05,"Bready Cricket Club, Magheramason",Londonderry,male,T20,international,Ireland,Hong Kong,Hong Kong,bat,Hong Kong
4,1007655,2016-06-18,Harare Sports Club,None,male,T20,international,Zimbabwe,India,India,field,Zimbabwe


In [377]:
matches_df['venue'].unique()

array(['Melbourne Cricket Ground', 'Simonds Stadium, South Geelong',
       'Adelaide Oval', 'Bready Cricket Club, Magheramason',
       'Harare Sports Club', 'McLean Park', 'Bay Oval', 'Eden Park',
       'The Rose Bowl', 'County Ground', 'Sophia Gardens',
       'Riverside Ground', 'Green Park',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'M Chinnaswamy Stadium',
       'Central Broward Regional Park Stadium Turf Ground',
       'Sinhalese Sports Club Ground',
       'Dubai International Cricket Stadium', 'Sheikh Zayed Stadium',
       'Saxton Oval', 'Asian Institute of Technology Ground',
       'Sydney Cricket Ground', 'Bellerive Oval', 'Westpac Stadium',
       'Seddon Park', 'Mangaung Oval', 'Senwes Park',
       'Kensington Oval, Bridgetown', "Queen's Park Oval, Port of Spain",
       'R Premadasa Stadium', 'North Sydney Oval', 'Manuka Oval',
       'Sabina Park, Kingston', 'R.Premadasa Stadium, Khettarama',
       'Gaddafi Stadium', 'JSCA International Stadium

In [378]:
deliveries_valid.head()

,match_id,innings_id,batting_team,bowling_team,over,ball,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,is_wicket,team_1,team_2,phase
431841,1375850,1,Australia,West Indies,0,1,DA Warner,AJ Hosein,JP Inglis,0,0,0,0,Australia,West Indies,pp
431842,1375850,1,Australia,West Indies,0,2,DA Warner,AJ Hosein,JP Inglis,0,0,0,0,Australia,West Indies,pp
431843,1375850,1,Australia,West Indies,0,3,DA Warner,AJ Hosein,JP Inglis,4,0,4,0,Australia,West Indies,pp
431844,1375850,1,Australia,West Indies,0,4,DA Warner,AJ Hosein,JP Inglis,1,0,1,0,Australia,West Indies,pp
431845,1375850,1,Australia,West Indies,0,5,JP Inglis,AJ Hosein,DA Warner,2,0,2,0,Australia,West Indies,pp


### Venue Metrics

In [379]:
deliveries_df['match_id'].nunique()

4972

In [380]:
deliveries_df['over']=deliveries_df['over']+1

In [381]:
deliveries_df['over'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22])

In [382]:
def phase(x):
    if x<=6:
        return "Power Play"
    elif x<=16:
        return "Middle Overs"
    else:
        return "Death Overs"

        

In [383]:
deliveries_df["phase"]=deliveries_df["over"].apply(phase)

In [384]:
# Total runs per innings per match
innings_totals = (
    deliveries_df
    .groupby(['match_id', 'innings_id', 'batting_team'])['runs_total']
    .sum()
    .reset_index(name='innings_runs')
)

In [385]:
innings_totals['innings_id'].value_counts()

innings_id
1    4972
2    4897
Name: count, dtype: int64

In [386]:
# Average first innings score
first_innings = innings_totals[innings_totals['innings_id'] == 1]

avg_first = (
    first_innings
    .merge(matches_df[['match_id', 'venue']], on='match_id',how='inner')
    .groupby('venue')['innings_runs']
    .mean()
    .reset_index(name='avg_first_innings_score')
)


In [387]:
avg_first['venue'].nunique()

446

In [388]:
# Average second innings score
second_innings = innings_totals[innings_totals['innings_id'] == 2]

avg_second = (
    second_innings
    .merge(matches_df[['match_id', 'venue']], on='match_id')
    .groupby('venue')['innings_runs']
    .mean()
    .reset_index(name='avg_second_innings_score')
)


In [389]:
# Average Power Play runs
pp_runs = (
    deliveries_df[deliveries_df['phase'] == 'Power Play']
    .groupby(['match_id', 'innings_id'])['runs_total']
    .sum()
    .reset_index(name='pp_runs')
)

avg_pp = (
    pp_runs
    .merge(matches_df[['match_id', 'venue']], on='match_id')
    .groupby('venue')['pp_runs']
    .mean()
    .reset_index(name='avg_pp_runs')
)


In [390]:
# Average death over runs
death_runs = (
    deliveries_df[deliveries_df['phase'] == 'Death Overs']
    .groupby(['match_id', 'innings_id'])['runs_total']
    .sum()
    .reset_index(name='death_runs')
)

avg_death = (
    death_runs
    .merge(matches_df[['match_id', 'venue']], on='match_id')
    .groupby('venue')['death_runs']
    .mean()
    .reset_index(name='avg_death_runs')
)


In [391]:
# Average Wickets per match
wickets = (
    deliveries_df
    .groupby(['match_id', 'innings_id'])['is_wicket']
    .sum()
    .reset_index(name='innings_wickets')
)

match_wickets = (
    wickets
    .groupby('match_id')['innings_wickets']
    .sum()
    .reset_index(name='total_wickets')
)

avg_wickets = (
    match_wickets
    .merge(matches_df[['match_id', 'venue']], on='match_id')
    .groupby('venue')['total_wickets']
    .mean()
    .reset_index(name='avg_wickets_per_match')
)


In [392]:
# Chasing Win percentage
# Get second innings batting team
second_innings_team = innings_totals[innings_totals['innings_id'] == 2][
    ['match_id', 'batting_team']
].rename(columns={'batting_team': 'chasing_team'})

chase_df = matches_df[['match_id', 'venue', 'winner']].merge(
    second_innings_team,
    on='match_id'
)

chase_df['chase_win'] = (
    chase_df['winner'] == chase_df['chasing_team']
).astype(int)

chasing_win_pct = (
    chase_df
    .groupby('venue')['chase_win']
    .mean()
    .reset_index(name='chasing_win_pct')
)


In [393]:
venue_stats= (
    avg_first
    .merge(avg_second, on='venue', how='left')
    .merge(avg_pp, on='venue', how='left')
    .merge(avg_death, on='venue', how='left')
    .merge(avg_wickets, on='venue', how='left')
    .merge(chasing_win_pct, on='venue', how='left')
)


In [394]:
len(venue_stats)

446

In [395]:
wc_2026_venues = [
    "Arun Jaitley Stadium", 
    "Wankhede Stadium", 
    "Pallekele International Cricket Stadium", 
    "Narendra Modi Stadium", 
    "Eden Gardens", 
    "MA Chidambaram Stadium, Chepauk", 
    "R Premadasa Stadium", 
    "Sinhalese Sports Club Ground"
]

In [396]:
venue_stats =  venue_stats[venue_stats['venue'].isin(wc_2026_venues)]

In [397]:
len(venue_stats)

8

In [398]:
venue_stats

,venue,avg_first_innings_score,avg_second_innings_score,avg_pp_runs,avg_death_runs,avg_wickets_per_match,chasing_win_pct
16,Arun Jaitley Stadium,175.000000,151.500000,22.500000,64.750000,10.000000,0.500000
98,Eden Gardens,142.285714,125.142857,22.428571,42.928571,12.285714,0.714286
210,"MA Chidambaram Stadium, Chepauk",150.500000,119.166667,25.500000,45.333333,12.166667,0.166667
246,Narendra Modi Stadium,170.600000,163.800000,27.600000,51.100000,10.400000,0.600000
271,Pallekele International Cricket Stadium,166.166667,146.533333,32.133333,49.736842,12.366667,0.400000
287,R Premadasa Stadium,141.696970,123.406250,25.153846,41.349206,12.757576,0.468750
337,Sinhalese Sports Club Ground,91.500000,98.000000,19.750000,33.333333,11.500000,1.000000
423,Wankhede Stadium,185.428571,177.000000,33.000000,60.714286,10.857143,0.714286


In [399]:
matches_df['venue'].nunique()

446

In [400]:
player_data.head()

,player,team,player_role,batting_position,batting_role,bowling_role
0,Aamir Kaleem,Oman,Allrounder,2.0,Opener,Fast Medium
1,Aarif Sheikh,Nepal,Allrounder,4.0,Top order,Part-time
2,Aasif Sheikh,Nepal,WicketKeeper,2.0,Opener,NaN
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin
4,Abrar Ahmed,Pakistan,Bowler,11.0,Tail,Spin


In [401]:
batting_stats.head()

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
19,Aryan Dutt,Netherlands,114,114,13,7,39,7,3,100.000000,16.285714,0.342105,0.087719,9,113.698630,75.609756,0.000000,Tail
61,Anrich Nortje,South Africa,17,14,3,2,7,0,1,82.352941,7.000000,0.411765,0.058824,9,100.000000,25.000000,0.000000,Tail
76,Alishan Sharafu,United Arab Emirates,1027,1386,44,38,382,135,61,134.956183,36.473684,0.371957,0.190847,1,148.275862,128.260870,138.022814,Opener
77,Aryansh Sharma,United Arab Emirates,134,165,7,7,56,16,6,123.134328,23.571429,0.417910,0.164179,1,175.000000,107.407407,125.242718,Opener
89,Adam Zampa,Australia,27,19,10,5,12,2,0,70.370370,3.800000,0.444444,0.074074,10,69.230769,100.000000,0.000000,Tail


In [402]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin
231,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN
253,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time
258,Tilak Varma,India,Batter,4.0,Top order,Spin
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium
246,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace
211,Rinku Singh,India,Batter,7.0,Finisher,Part-time
271,Washington Sundar,India,Allrounder,7.0,Finisher,Spin
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin


In [403]:
batting_stats[batting_stats['team']=='India'].sort_values(by='batting_position')

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
193,Abhishek Sharma,India,691,1297,37,33,215,123,88,187.698987,39.303030,0.311143,0.305355,1,204.761905,196.913580,184.055118,Opener
1929,Sanju Samson,India,467,704,27,27,162,57,45,150.749465,26.074074,0.346895,0.218415,1,142.307692,171.348315,137.130802,Opener
847,Ishan Kishan,India,99,215,4,4,26,22,16,217.171717,53.750000,0.262626,0.383838,2,271.428571,248.387097,195.081967,Opener
1877,Suryakumar Yadav,India,571,836,40,35,198,79,43,146.409807,23.885714,0.346760,0.213660,4,142.857143,158.333333,132.400000,Top order
2138,Tilak Varma,India,601,847,22,14,168,62,45,140.931780,60.500000,0.279534,0.178037,4,167.187500,139.000000,124.855491,Top order
752,Hardik Pandya,India,464,691,28,19,144,56,40,148.922414,36.368421,0.310345,0.206897,5,171.555556,124.553571,173.333333,Middle order
1797,Shivam Dube,India,324,470,25,17,95,30,30,145.061728,27.647059,0.293210,0.185185,6,161.764706,140.186916,62.500000,Middle order
2233,Washington Sundar,India,121,147,8,5,48,11,7,121.487603,29.400000,0.396694,0.148760,7,119.354839,123.728814,0.000000,Finisher
1716,Rinku Singh,India,208,285,17,11,69,23,16,137.019231,25.909091,0.331731,0.187500,7,161.616162,106.153846,127.272727,Finisher
145,Axar Patel,India,267,313,21,13,81,19,12,117.228464,24.076923,0.303371,0.116105,8,122.105263,117.272727,109.677419,Finisher


In [404]:
batting_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 19 to 2283
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player              279 non-null    object 
 1   team                279 non-null    object 
 2   balls_faced         279 non-null    int64  
 3   runs                279 non-null    int64  
 4   matches_played      279 non-null    int64  
 5   dismissals          279 non-null    int64  
 6   dot_balls           279 non-null    int64  
 7   fours               279 non-null    int64  
 8   sixes               279 non-null    int64  
 9   strike_rate         279 non-null    float64
 10  batting_average     279 non-null    float64
 11  dot_ball_pct        279 non-null    float64
 12  boundary_pct        279 non-null    float64
 13  batting_position    279 non-null    int64  
 14  death_strike_rate   279 non-null    float64
 15  middle_strike_rate  279 non-null    float64
 16  pp_strike_r

In [405]:
bowling_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 16 to 1641
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player               205 non-null    object 
 1   team                 205 non-null    object 
 2   balls_bowled         205 non-null    int64  
 3   runs_conceded        205 non-null    int64  
 4   matches_played       205 non-null    int64  
 5   dot_balls            205 non-null    int64  
 6   wickets              205 non-null    int64  
 7   overs_bowled         205 non-null    float64
 8   economy              205 non-null    float64
 9   dot_ball_pct         205 non-null    float64
 10  bowling_average      205 non-null    float64
 11  bowling_strike_rate  205 non-null    float64
 12  death_economy        205 non-null    float64
 13  middle_economy       205 non-null    float64
 14  pp_economy           205 non-null    float64
 15  bowling_role         205 non-null    object

In [406]:
venue_stats.head()

,venue,avg_first_innings_score,avg_second_innings_score,avg_pp_runs,avg_death_runs,avg_wickets_per_match,chasing_win_pct
16,Arun Jaitley Stadium,175.000000,151.500000,22.500000,64.750000,10.000000,0.500000
98,Eden Gardens,142.285714,125.142857,22.428571,42.928571,12.285714,0.714286
210,"MA Chidambaram Stadium, Chepauk",150.500000,119.166667,25.500000,45.333333,12.166667,0.166667
246,Narendra Modi Stadium,170.600000,163.800000,27.600000,51.100000,10.400000,0.600000
271,Pallekele International Cricket Stadium,166.166667,146.533333,32.133333,49.736842,12.366667,0.400000


In [407]:
batting_stats.head()

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,batting_average,dot_ball_pct,boundary_pct,batting_position,death_strike_rate,middle_strike_rate,pp_strike_rate,batting_role
19,Aryan Dutt,Netherlands,114,114,13,7,39,7,3,100.000000,16.285714,0.342105,0.087719,9,113.698630,75.609756,0.000000,Tail
61,Anrich Nortje,South Africa,17,14,3,2,7,0,1,82.352941,7.000000,0.411765,0.058824,9,100.000000,25.000000,0.000000,Tail
76,Alishan Sharafu,United Arab Emirates,1027,1386,44,38,382,135,61,134.956183,36.473684,0.371957,0.190847,1,148.275862,128.260870,138.022814,Opener
77,Aryansh Sharma,United Arab Emirates,134,165,7,7,56,16,6,123.134328,23.571429,0.417910,0.164179,1,175.000000,107.407407,125.242718,Opener
89,Adam Zampa,Australia,27,19,10,5,12,2,0,70.370370,3.800000,0.444444,0.074074,10,69.230769,100.000000,0.000000,Tail


In [431]:
import pandas as pd
import numpy as np

# Make a copy
batting_scaled = batting_stats.copy()

# ----------------------------
# Columns to Scale
# ----------------------------
scale_cols = [
    'matches_played',
    'strike_rate',
    'batting_average',
    'boundary_pct',
    'death_strike_rate',
    'middle_strike_rate',
    'pp_strike_rate',
    'dot_ball_pct'
]

# ----------------------------
# Min-Max Scaling Function
# ----------------------------
def minmax_scale_team(group):
    
    for col in scale_cols:
        min_val = group[col].min()
        max_val = group[col].max()
        
        # Avoid division by zero if all values same
        if max_val - min_val == 0:
            group[col + '_scaled'] = 0.5  # neutral value
        else:
            group[col + '_scaled'] = (group[col] - min_val) / (max_val - min_val)
    
    return group


# ----------------------------
# Apply Scaling Within Each Team
# ----------------------------
batting_scaled = (
    cdf
    .groupby('team', group_keys=False)
    .apply(minmax_scale_team)
)

# ----------------------------
# Invert dot_ball_pct (lower is better)
# ----------------------------
batting_scaled['dot_ball_pct_scaled'] = 1 - batting_scaled['dot_ball_pct_scaled']


# ----------------------------
# Final Scaled Columns Available:
# ----------------------------
scaled_columns = [col + '_scaled' for col in scale_cols]


batting_scaled.head()


C:\Users\Adnan Khader\AppData\Local\Temp\ipykernel_9804\2407368574.py:45: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(minmax_scale_team)


,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,...,pp_strike_rate,batting_role,strike_rate_scaled,batting_average_scaled,boundary_pct_scaled,death_strike_rate_scaled,middle_strike_rate_scaled,pp_strike_rate_scaled,dot_ball_pct_scaled,matches_played_scaled
19,Aryan Dutt,Netherlands,114,114,13,7,39,7,3,100.000000,...,0.000000,Tail,0.323418,0.376660,0.313544,0.420286,0.321902,0.000000,0.311872,0.384615
61,Anrich Nortje,South Africa,17,14,3,2,7,0,1,82.352941,...,0.000000,Tail,0.253114,0.000000,0.134217,0.516129,0.062500,0.000000,0.429599,0.000000
76,Alishan Sharafu,United Arab Emirates,1027,1386,44,38,382,135,61,134.956183,...,138.022814,Opener,0.880727,1.000000,0.960849,0.655172,0.807035,1.000000,0.563389,0.955556
77,Aryansh Sharma,United Arab Emirates,134,165,7,7,56,16,6,123.134328,...,125.242718,Opener,0.716832,0.602686,0.826585,0.833333,0.675822,0.907406,0.361194,0.133333
89,Adam Zampa,Australia,27,19,10,5,12,2,0,70.370370,...,0.000000,Tail,0.274996,0.046154,0.260842,0.356292,0.598131,0.000000,0.624309,0.285714


In [432]:
batting_scaled[batting_scaled['team']=='India'].sort_values(by='batting_position')

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,...,pp_strike_rate,batting_role,strike_rate_scaled,batting_average_scaled,boundary_pct_scaled,death_strike_rate_scaled,middle_strike_rate_scaled,pp_strike_rate_scaled,dot_ball_pct_scaled,matches_played_scaled
193,Abhishek Sharma,India,691,1297,37,33,215,123,88,187.698987,...,184.055118,Opener,0.853008,0.643748,0.795529,0.738318,0.792769,0.943476,0.914988,0.921053
1929,Sanju Samson,India,467,704,27,27,162,57,45,150.749465,...,137.130802,Opener,0.668725,0.421413,0.569030,0.493170,0.689844,0.702939,0.852343,0.657895
847,Ishan Kishan,India,99,215,4,4,26,22,16,217.171717,...,195.081967,Opener,1.000000,0.886555,1.000000,1.000000,1.000000,1.000000,1.000000,0.052632
1877,Suryakumar Yadav,India,571,836,40,35,198,79,43,146.409807,...,132.400000,Top order,0.647082,0.384634,0.556641,0.495327,0.637446,0.678689,0.852580,1.000000
2138,Tilak Varma,India,601,847,22,14,168,62,45,140.931780,...,124.855491,Top order,0.619761,1.000000,0.463832,0.590829,0.559610,0.640016,0.970374,0.526316
752,Hardik Pandya,India,464,691,28,19,144,56,40,148.922414,...,173.333333,Middle order,0.659613,0.594427,0.539020,0.607975,0.501449,0.888515,0.916387,0.684211
1797,Shivam Dube,India,324,470,25,17,95,30,30,145.061728,...,62.500000,Middle order,0.640358,0.447850,0.482456,0.569544,0.564389,0.320378,0.946411,0.605263
2233,Washington Sundar,India,121,147,8,5,48,11,7,121.487603,...,0.000000,Finisher,0.522785,0.477311,0.387560,0.403075,0.498129,0.000000,0.765084,0.157895
1716,Rinku Singh,India,208,285,17,11,69,23,16,137.019231,...,127.272727,Finisher,0.600247,0.418640,0.488487,0.568961,0.427373,0.652406,0.878914,0.394737
145,Axar Patel,India,267,313,21,13,81,19,12,117.228464,...,109.677419,Finisher,0.501542,0.387847,0.302484,0.413871,0.472137,0.562212,0.928607,0.500000


In [433]:
bowling_stats.columns

Index(['player', 'team', 'balls_bowled', 'runs_conceded', 'matches_played',
       'dot_balls', 'wickets', 'overs_bowled', 'economy', 'dot_ball_pct',
       'bowling_average', 'bowling_strike_rate', 'death_economy',
       'middle_economy', 'pp_economy', 'bowling_role'],
      dtype='object')

In [434]:
import pandas as pd
import numpy as np

# Copy
bowling_scaled = bowling_stats.copy()

# -----------------------------
# Columns Classification
# -----------------------------

higher_better_cols = [
    'wickets',
    'dot_ball_pct',
    'matches_played'
]

lower_better_cols = [
    'economy',
    'death_economy',
    'middle_economy',
    'pp_economy',
    'bowling_average',
    'bowling_strike_rate'
]

# -----------------------------
# Min-Max Scaling Function
# -----------------------------

def minmax_scale_team(group):
    
    # Higher is better
    for col in higher_better_cols:
        min_val = group[col].min()
        max_val = group[col].max()
        
        if max_val - min_val == 0:
            group[col + '_scaled'] = 0.5
        else:
            group[col + '_scaled'] = (group[col] - min_val) / (max_val - min_val)
    
    # Lower is better
    for col in lower_better_cols:
        min_val = group[col].min()
        max_val = group[col].max()
        
        if max_val - min_val == 0:
            group[col + '_scaled'] = 0.5
        else:
            group[col + '_scaled'] = 1 - ((group[col] - min_val) / (max_val - min_val))
    
    return group


# -----------------------------
# Apply Scaling Within Each Team
# -----------------------------

bowling_scaled = (
    bowling_scaled
    .groupby('team', group_keys=False)
    .apply(minmax_scale_team)
)

# -----------------------------
# List Created Scaled Columns
# -----------------------------

scaled_cols = [col + '_scaled' for col in higher_better_cols + lower_better_cols]

print("Scaled Bowling Columns:")
print(scaled_cols)

bowling_scaled.head()


Scaled Bowling Columns:
['wickets_scaled', 'dot_ball_pct_scaled', 'matches_played_scaled', 'economy_scaled', 'death_economy_scaled', 'middle_economy_scaled', 'pp_economy_scaled', 'bowling_average_scaled', 'bowling_strike_rate_scaled']


C:\Users\Adnan Khader\AppData\Local\Temp\ipykernel_9804\3386746257.py:62: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(minmax_scale_team)


,player,team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,...,bowling_role,wickets_scaled,dot_ball_pct_scaled,matches_played_scaled,economy_scaled,death_economy_scaled,middle_economy_scaled,pp_economy_scaled,bowling_average_scaled,bowling_strike_rate_scaled
16,Aryan Dutt,Netherlands,449,501,21,187,22,74.833333,6.694878,0.416481,...,Spin,0.724138,0.691882,1.000000,0.694372,0.333750,0.515323,0.347163,0.606756,0.551610
39,Anrich Nortje,South Africa,303,389,13,120,14,50.500000,7.702970,0.396040,...,Fast,0.583333,0.879042,0.666667,0.979239,0.108201,1.000000,0.000000,0.450875,0.348107
49,Alishan Sharafu,United Arab Emirates,1,3,1,0,0,0.166667,18.000000,0.000000,...,Spin,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
60,Adam Zampa,Australia,776,1050,34,253,55,129.333333,8.118557,0.326031,...,Spin,1.000000,0.279936,1.000000,0.473119,0.419178,0.713460,0.174917,0.962952,0.967161
80,Graeme Cremer,Zimbabwe,42,46,2,18,2,7.000000,6.571429,0.428571,...,Spin,0.044444,0.871910,0.000000,0.805559,0.900000,0.566025,1.000000,0.592920,0.513043


In [435]:
bowling_stats[bowling_stats['player']=='Arshdeep Singh']

,player,team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role
201,Arshdeep Singh,India,694,942,30,279,55,115.666667,8.144092,0.402017,17.127273,12.618182,8.610879,5.731343,8.273196,Fast Medium


In [436]:
batting_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 19 to 2283
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player                     279 non-null    object 
 1   team                       279 non-null    object 
 2   balls_faced                279 non-null    int64  
 3   runs                       279 non-null    int64  
 4   matches_played             279 non-null    int64  
 5   dismissals                 279 non-null    int64  
 6   dot_balls                  279 non-null    int64  
 7   fours                      279 non-null    int64  
 8   sixes                      279 non-null    int64  
 9   strike_rate                279 non-null    float64
 10  batting_average            279 non-null    float64
 11  dot_ball_pct               279 non-null    float64
 12  boundary_pct               279 non-null    float64
 13  batting_position           279 non-null    int64  
 1

### Scoring Batsmen based on role

In [437]:
def compute_batting_role_score(row):
    
    role = row['batting_role']
    
    # Opener formula
    if role == 'Opener':
        score = (
            0.35 * row['pp_strike_rate_scaled'] +
            0.25 * row['strike_rate_scaled'] +
            0.20 * row['boundary_pct_scaled'] +
            0.20 * row['dot_ball_pct_scaled']
        )
    
    # Top Order + Middle Order (same logic)
    elif role in ['Top order', 'Middle order']:
        score = (
            0.30 * row['batting_average_scaled'] +
            0.25 * row['strike_rate_scaled'] +
            0.20 * row['middle_strike_rate_scaled'] +
            0.15 * row['dot_ball_pct_scaled'] +
            0.10 * row['boundary_pct_scaled']
        )
    
    # Finisher formula
    elif role == 'Finisher':
        score = (
            0.40 * row['death_strike_rate_scaled'] +
            0.25 * row['boundary_pct_scaled'] +
            0.20 * row['strike_rate_scaled'] +
            0.15 * row['dot_ball_pct_scaled']
        )
    
    # Tail (batting is not primary skill)
    elif role == 'Tail':
        score = (
            0.10 * row['strike_rate_scaled'] +
            0.10 * row['boundary_pct_scaled']
        )
    
    # Safety fallback
    else:
        score = 0.0
    
    return score


In [438]:
batting_scaled['batting_role_score'] = batting_scaled.apply(
    compute_batting_role_score,
    axis=1
)


In [439]:
batting_scaled.head()

,player,team,balls_faced,runs,matches_played,dismissals,dot_balls,fours,sixes,strike_rate,...,batting_role,strike_rate_scaled,batting_average_scaled,boundary_pct_scaled,death_strike_rate_scaled,middle_strike_rate_scaled,pp_strike_rate_scaled,dot_ball_pct_scaled,matches_played_scaled,batting_role_score
19,Aryan Dutt,Netherlands,114,114,13,7,39,7,3,100.000000,...,Tail,0.323418,0.376660,0.313544,0.420286,0.321902,0.000000,0.311872,0.384615,0.063696
61,Anrich Nortje,South Africa,17,14,3,2,7,0,1,82.352941,...,Tail,0.253114,0.000000,0.134217,0.516129,0.062500,0.000000,0.429599,0.000000,0.038733
76,Alishan Sharafu,United Arab Emirates,1027,1386,44,38,382,135,61,134.956183,...,Opener,0.880727,1.000000,0.960849,0.655172,0.807035,1.000000,0.563389,0.955556,0.875029
77,Aryansh Sharma,United Arab Emirates,134,165,7,7,56,16,6,123.134328,...,Opener,0.716832,0.602686,0.826585,0.833333,0.675822,0.907406,0.361194,0.133333,0.734356
89,Adam Zampa,Australia,27,19,10,5,12,2,0,70.370370,...,Tail,0.274996,0.046154,0.260842,0.356292,0.598131,0.000000,0.624309,0.285714,0.053584


In [440]:
player_data = pd.merge(player_data,batting_scaled[['player','batting_role_score']],on='player',how='left')

In [441]:
player_data

,player,team,player_role,batting_position,batting_role,bowling_role,batting_role_score_x,bowling_role_score,batting_role_score_y
0,Aamir Kaleem,Oman,Allrounder,2.0,Opener,Fast Medium,0.654279,0.700872,0.654279
1,Aarif Sheikh,Nepal,Allrounder,4.0,Top order,Part-time,0.407962,0.027481,0.407962
2,Aasif Sheikh,Nepal,WicketKeeper,2.0,Opener,NaN,0.459684,NaN,0.459684
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.885572,0.097194,0.885572
4,Abrar Ahmed,Pakistan,Bowler,11.0,Tail,Spin,0.000000,0.697939,0.000000
...,...,...,...,...,...,...,...,...,...
282,Xavier Bartlett,Australia,Bowler,8.0,Finisher,Fast Medium,0.514023,0.494616,0.514023
283,Yuvraj Samra,Canada,Batter,1.0,Opener,Part-time,0.903848,0.336884,0.903848
284,Zach Lion-Cachet,Netherlands,Batter,3.0,Top order,Part-time,0.253706,0.393344,0.253706
285,Zain Ali,Italy,Bowler,7.0,Finisher,Fast Medium,0.000000,0.666071,0.000000


In [442]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role,batting_role_score_x,bowling_role_score,batting_role_score_y
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.885572,0.097194,0.885572
237,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,0.697485,NaN,0.697485
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,1.000000,NaN,1.000000
259,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.588201,0.592310,0.588201
264,Tilak Varma,India,Batter,4.0,Top order,Spin,0.758802,0.094509,0.758802
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.634881,0.368544,0.634881
252,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace,0.597530,0.187162,0.597530
217,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.601593,0.577418,0.601593
277,Washington Sundar,India,Allrounder,7.0,Finisher,Spin,0.477440,0.415291,0.477440
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.480769,0.390545,0.480769


In [443]:
bowling_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 16 to 1641
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   player                      205 non-null    object 
 1   team                        205 non-null    object 
 2   balls_bowled                205 non-null    int64  
 3   runs_conceded               205 non-null    int64  
 4   matches_played              205 non-null    int64  
 5   dot_balls                   205 non-null    int64  
 6   wickets                     205 non-null    int64  
 7   overs_bowled                205 non-null    float64
 8   economy                     205 non-null    float64
 9   dot_ball_pct                205 non-null    float64
 10  bowling_average             205 non-null    float64
 11  bowling_strike_rate         205 non-null    float64
 12  death_economy               205 non-null    float64
 13  middle_economy              205 non-nu

### Scoring Bowlers based on role

In [444]:
def compute_bowling_role_score(row):
    
    role = row['bowling_role']
    
    # Fast / Fast Medium bowlers (PP + Death specialists)
    if role in ['Fast', 'Fast Medium', 'Medium Pace']:
        score = (
            0.30 * row['economy_scaled'] +
            0.25 * row['death_economy_scaled'] +
            0.20 * row['wickets_scaled'] +
            0.15 * row['dot_ball_pct_scaled'] +
            0.10 * row['matches_played_scaled']
        )
    
    # Spin bowlers (Middle overs control)
    elif role == 'Spin':
        score = (
            0.35 * row['middle_economy_scaled'] +
            0.25 * row['dot_ball_pct_scaled'] +
            0.20 * row['wickets_scaled'] +
            0.10 * row['economy_scaled'] +
            0.10 * row['matches_played_scaled']
        )
    
    # Part-time / Unknown role (fallback)
    else:
        score = (
            0.30 * row['economy_scaled'] +
            0.25 * row['wickets_scaled'] +
            0.20 * row['dot_ball_pct_scaled'] +
            0.15 * row['bowling_average_scaled'] +
            0.10 * row['matches_played_scaled']
        )
    
    return score


In [445]:
bowling_scaled['bowling_role_score'] = bowling_scaled.apply(
    compute_bowling_role_score,
    axis=1
)


In [446]:
bowling_scaled.head()

,player,team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,...,wickets_scaled,dot_ball_pct_scaled,matches_played_scaled,economy_scaled,death_economy_scaled,middle_economy_scaled,pp_economy_scaled,bowling_average_scaled,bowling_strike_rate_scaled,bowling_role_score
16,Aryan Dutt,Netherlands,449,501,21,187,22,74.833333,6.694878,0.416481,...,0.724138,0.691882,1.000000,0.694372,0.333750,0.515323,0.347163,0.606756,0.551610,0.667598
39,Anrich Nortje,South Africa,303,389,13,120,14,50.500000,7.702970,0.396040,...,0.583333,0.879042,0.666667,0.979239,0.108201,1.000000,0.000000,0.450875,0.348107,0.636012
49,Alishan Sharafu,United Arab Emirates,1,3,1,0,0,0.166667,18.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.350000
60,Adam Zampa,Australia,776,1050,34,253,55,129.333333,8.118557,0.326031,...,1.000000,0.279936,1.000000,0.473119,0.419178,0.713460,0.174917,0.962952,0.967161,0.667007
80,Graeme Cremer,Zimbabwe,42,46,2,18,2,7.000000,6.571429,0.428571,...,0.044444,0.871910,0.000000,0.805559,0.900000,0.566025,1.000000,0.592920,0.513043,0.505531


In [447]:
player_data = pd.merge(player_data,bowling_scaled[['player','bowling_role_score']],on='player',how='left')

In [448]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role,batting_role_score_x,bowling_role_score_x,batting_role_score_y,bowling_role_score_y
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.885572,0.097194,0.885572,0.097194
237,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,0.697485,NaN,0.697485,NaN
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,1.000000,NaN,1.000000,NaN
259,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.588201,0.592310,0.588201,0.592310
264,Tilak Varma,India,Batter,4.0,Top order,Spin,0.758802,0.094509,0.758802,0.094509
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.634881,0.368544,0.634881,0.368544
252,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace,0.597530,0.187162,0.597530,0.187162
217,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.601593,0.577418,0.601593,0.577418
277,Washington Sundar,India,Allrounder,7.0,Finisher,Spin,0.477440,0.415291,0.477440,0.415291
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.480769,0.390545,0.480769,0.390545


### Adding experience as a factor to score

In [449]:
bowling_scaled['matches_played_scaled']

16      1.000000
39      0.666667
49      0.000000
60      1.000000
80      0.000000
          ...   
1605    0.090909
1606    0.468750
1615    0.000000
1630    0.421053
1641    0.000000
Name: matches_played_scaled, Length: 205, dtype: float64

In [450]:
batting_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 19 to 2283
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player                     279 non-null    object 
 1   team                       279 non-null    object 
 2   balls_faced                279 non-null    int64  
 3   runs                       279 non-null    int64  
 4   matches_played             279 non-null    int64  
 5   dismissals                 279 non-null    int64  
 6   dot_balls                  279 non-null    int64  
 7   fours                      279 non-null    int64  
 8   sixes                      279 non-null    int64  
 9   strike_rate                279 non-null    float64
 10  batting_average            279 non-null    float64
 11  dot_ball_pct               279 non-null    float64
 12  boundary_pct               279 non-null    float64
 13  batting_position           279 non-null    int64  
 1

In [453]:
batting_scaled['batting_role_score'] = batting_scaled['batting_role_score']*0.85 + batting_scaled['matches_played_scaled']*0.15

In [456]:
bowling_scaled['bowling_role_score'] = bowling_scaled['bowling_role_score']*0.85 + bowling_scaled['matches_played_scaled']*0.15

In [463]:
player_data = pd.merge(player_data,bowling_scaled[['player','bowling_role_score']],on='player',how='left')

In [464]:
player_data = pd.merge(player_data,batting_scaled[['player','batting_role_score']],on='player',how='left')

In [465]:
player_data.head()

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score
0,Aamir Kaleem,Oman,Allrounder,2.0,Opener,Fast Medium,0.663923,0.689470
1,Aarif Sheikh,Nepal,Allrounder,4.0,Top order,Part-time,0.023359,0.380101
2,Aasif Sheikh,Nepal,WicketKeeper,2.0,Opener,NaN,NaN,0.519898
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.155342,0.890894
4,Abrar Ahmed,Pakistan,Bowler,11.0,Tail,Spin,0.717533,0.006977


In [466]:
player_data.columns

Index(['player', 'team', 'player_role', 'batting_position', 'batting_role',
       'bowling_role', 'bowling_role_score', 'batting_role_score'],
      dtype='object')

In [467]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.155342,0.890894
261,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,NaN,0.691546
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,NaN,0.857895
283,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.503464,0.649971
288,Tilak Varma,India,Batter,4.0,Top order,Spin,0.089423,0.723929
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.463262,0.642281
276,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace,0.236360,0.598690
241,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.495351,0.570564
301,Washington Sundar,India,Allrounder,7.0,Finisher,Spin,0.407543,0.429508
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.463781,0.483654


In [468]:
player_data['bowling_role_score'].fillna(value=0,inplace=True)

C:\Users\Adnan Khader\AppData\Local\Temp\ipykernel_9804\3536601625.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  player_data['bowling_role_score'].fillna(value=0,inplace=True)


In [469]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.155342,0.890894
261,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,0.000000,0.691546
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,0.000000,0.857895
283,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.503464,0.649971
288,Tilak Varma,India,Batter,4.0,Top order,Spin,0.089423,0.723929
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.463262,0.642281
276,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace,0.236360,0.598690
241,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.495351,0.570564
301,Washington Sundar,India,Allrounder,7.0,Finisher,Spin,0.407543,0.429508
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.463781,0.483654


### Assigning Captain

In [471]:
captains_list = [
    "Suryakumar Yadav",
    "Jatinder Singh",
    "Dilpreet Bajwa",
    "Harry Brook",
    "Wayne Madsen",
    "Gerhard Erasmus",
    "Rohit Paudel",
    "Mitchell Santner",
    "Richie Berrington",
    "Aiden Markram",
    "Monank Patel",
    "Shai Hope",
    "Dasun Shanaka",
    "Mitchell Marsh",
    "Paul Stirling",
    "Scott Edwards",
    "Salman Ali Agha",
    "Muhammad Waseem",
    "Sikandar Raza"
]


In [472]:
player_data['role_in_team']=player_data['player'].apply(lambda x: 'captain' if x in captains_list else 'member')

In [485]:
player_data[player_data['team']=='India'].sort_values(by='batting_position')

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.155342,0.890894,member
261,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,0.000000,0.691546,member
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,0.000000,0.857895,member
283,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.503464,0.649971,captain
288,Tilak Varma,India,Batter,4.0,Top order,Spin,0.089423,0.723929,member
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.463262,0.642281,member
276,Shivam Dube,India,Allrounder,6.0,Middle order,Medium Pace,0.236360,0.598690,member
241,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.495351,0.570564,member
301,Washington Sundar,India,Allrounder,7.0,Finisher,Spin,0.407543,0.429508,member
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.463781,0.483654,member


In [475]:
len(player_data[player_data['role_in_team']=='captain'])

19

### Deriving playing 11

In [566]:
def select_playing_xi_from_player_data(team_df, min_bowlers=5):
    
    df = team_df.copy()
    selected = []

    # ----------------------------------
    # 1️⃣ Select Captain (Mandatory)
    # ----------------------------------
    captain_df = df[df['role_in_team'] == 'captain']
    if not captain_df.empty:
        captain = captain_df.iloc[0]  # usually only one captain
        selected.append(captain['player'])
    
    df = df[~df['player'].isin(selected)]

    # ----------------------------------
    # 2️⃣ Select Wicketkeeper (Mandatory)
    # ----------------------------------
    wk_df = df[df['player_role'] == 'WicketKeeper']
    if not wk_df.empty:
        wk = wk_df.sort_values('batting_role_score', ascending=False).iloc[0]
        selected.append(wk['player'])
    
    df = df[~df['player'].isin(selected)]

    # ----------------------------------
    # 3️⃣ Select 2 Openers (Best batting role score)
    # ----------------------------------
    openers = (
        df[df['batting_role'] == 'Opener']
        .sort_values('batting_role_score', ascending=False)
        .head(2)
    )
    selected.extend(openers['player'].tolist())
    df = df[~df['player'].isin(selected)]

    # ----------------------------------
    # 4️⃣ Select Top Order (1–2 players)
    # ----------------------------------
    top_order = (
        df[df['batting_role'] == 'Top order']
        .sort_values('batting_role_score', ascending=False)
        .head(2)
    )
    selected.extend(top_order['player'].tolist())
    df = df[~df['player'].isin(selected)]

    # ----------------------------------
    # 5️⃣ Select 1 Best Finisher
    # ----------------------------------
    finisher = (
        df[df['batting_role'] == 'Finisher']
        .sort_values('batting_role_score', ascending=False)
        .head(1)
    )
    selected.extend(finisher['player'].tolist())
    df = df[~df['player'].isin(selected)]

    # ----------------------------------
    # 6️⃣ Ensure Minimum Bowling Options
    # ----------------------------------
    def count_bowling(players):
        subset = team_df[team_df['player'].isin(players)]
        return subset[
            subset['player_role'].isin(['Bowler', 'Allrounder'])
        ].shape[0]

    while count_bowling(selected) < min_bowlers:
        best_bowler = (
            df[df['player_role'].isin(['Bowler', 'Allrounder'])]
            .sort_values('bowling_role_score', ascending=False)
            .head(1)
        )
        if best_bowler.empty:
            break
        
        player_name = best_bowler.iloc[0]['player']
        selected.append(player_name)
        df = df[df['player'] != player_name]

    # ----------------------------------
    # 7️⃣ Fill Remaining Slots (Best Allround Balance)
    # ----------------------------------
    remaining_slots = 11 - len(selected)

    remaining_players = df.sort_values(
        ['batting_role_score', 'bowling_role_score'],
        ascending=False
    ).head(remaining_slots)

    selected.extend(remaining_players['player'].tolist())

    # Final XI sorted by batting position (realistic lineup)
    playing_xi = team_df[team_df['player'].isin(selected)].sort_values('batting_position')

    return playing_xi


In [567]:
india_df = player_data[player_data['team'] == 'India'].copy()

playing_xi = select_playing_xi_from_player_data(india_df)

playing_xi


,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
3,Abhishek Sharma,India,Allrounder,1.0,Opener,Spin,0.155342,0.890894,member
261,Sanju Samson,India,WicketKeeper,1.0,Opener,NaN,0.000000,0.691546,member
97,Ishan Kishan,India,WicketKeeper,2.0,Opener,NaN,0.000000,0.857895,member
283,Suryakumar Yadav,India,Batter,4.0,Top order,Part-time,0.503464,0.649971,captain
288,Tilak Varma,India,Batter,4.0,Top order,Spin,0.089423,0.723929,member
89,Hardik Pandya,India,Allrounder,5.0,Middle order,Fast Medium,0.463262,0.642281,member
241,Rinku Singh,India,Batter,7.0,Finisher,Part-time,0.495351,0.570564,member
20,Axar Patel,India,Allrounder,8.0,Finisher,Spin,0.463781,0.483654,member
16,Arshdeep Singh,India,Bowler,9.0,Tail,Fast Medium,0.503323,0.109277,member
116,Jasprit Bumrah,India,Bowler,10.0,Tail,Fast,0.518257,0.094637,member


In [568]:
all_teams = player_data['team'].unique()

playing_xi_list = []

for team in all_teams:
    team_df = player_data[player_data['team'] == team].copy()
    
    xi = select_playing_xi_from_player_data(team_df)
    
    xi['team'] = team  # ensure team column exists
    playing_xi_list.append(xi)

# Combine all teams XI into one table
playing_xi_table = pd.concat(playing_xi_list, ignore_index=True)

In [569]:
len(playing_xi_table.sort_values(by=['team','batting_position']))

240

In [570]:
t20_world_cup_2026_teams = [
    "Australia", 
    "Canada", 
    "England", 
    "India", 
    "Ireland", 
    "Italy", 
    "Namibia", 
    "Nepal", 
    "Netherlands", 
    "New Zealand", 
    "Oman", 
    "Pakistan", 
    "Scotland", 
    "South Africa", 
    "Sri Lanka", 
    "United Arab Emirates", 
    "United States of America", 
    "West Indies", 
    "Zimbabwe"
]

In [571]:
for i in t20_world_cup_2026_teams:
    print(i, ':', len(playing_xi_table[playing_xi_table['team'] == i]))


Australia : 11
Canada : 11
England : 11
India : 11
Ireland : 11
Italy : 11
Namibia : 11
Nepal : 11
Netherlands : 11
New Zealand : 11
Oman : 11
Pakistan : 11
Scotland : 42
South Africa : 11
Sri Lanka : 11
United Arab Emirates : 11
United States of America : 11
West Indies : 11
Zimbabwe : 11


In [573]:
playing_xi_table[playing_xi_table['team'] == 'Scotland'].drop_duplicates()

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
132,George Munsey,Scotland,Batter,1.0,Opener,NaN,0.000000,0.940970,member
133,Michael Jones,Scotland,Batter,2.0,Opener,Part-time,0.147955,0.818908,member
134,Michael Jones,Scotland,Batter,2.0,Opener,Part-time,0.147955,0.155833,member
149,Oliver Hairs,Scotland,Batter,2.0,Opener,NaN,0.000000,0.708739,member
150,Brandon McMullen,Scotland,Allrounder,3.0,Top order,Fast Medium,0.517375,0.666292,member
151,Charlie Tear,Scotland,WicketKeeper,3.0,Top order,NaN,0.000000,0.409626,member
152,Richie Berrington,Scotland,Batter,4.0,Top order,NaN,0.000000,0.558091,captain
153,Matthew Cross,Scotland,WicketKeeper,5.0,Middle order,NaN,0.000000,0.553489,member
154,Michael Leask,Scotland,Allrounder,6.0,Middle order,Spin,0.715684,0.569748,member
155,Charlie McBride,Scotland,Allrounder,8.0,Finisher,NaN,0.000000,0.768506,member


In [574]:
scotland_xi = playing_xi_table[playing_xi_table['team'] == 'Scotland'].drop_duplicates().drop(index=[134, 157, 158])

In [575]:
playing_xi_table = playing_xi_table[playing_xi_table['team'] != 'Scotland']

In [576]:
playing_xi_table = pd.concat([playing_xi_table, scotland_xi], ignore_index=True)

In [577]:
for i in t20_world_cup_2026_teams:
    print(i, ':', len(playing_xi_table[playing_xi_table['team'] == i]))

Australia : 11
Canada : 11
England : 11
India : 11
Ireland : 11
Italy : 11
Namibia : 11
Nepal : 11
Netherlands : 11
New Zealand : 11
Oman : 11
Pakistan : 11
Scotland : 11
South Africa : 11
Sri Lanka : 11
United Arab Emirates : 11
United States of America : 11
West Indies : 11
Zimbabwe : 11


In [578]:
playing_xi_table[playing_xi_table['team'] == 'Australia' ]

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
0,Mitchell Marsh,Australia,Allrounder,1.0,Opener,Fast Medium,0.444949,0.798271,captain
1,Travis Head,Australia,Batter,2.0,Opener,NaN,0.000000,0.853521,member
2,Josh Inglis,Australia,WicketKeeper,3.0,Top order,NaN,0.000000,0.794044,member
3,Cameron Green,Australia,Allrounder,4.0,Top order,Fast Medium,0.206670,0.768919,member
4,Glenn Maxwell,Australia,Allrounder,4.0,Top order,Spin,0.230558,0.819641,member
5,Matthew Renshaw,Australia,Batter,4.0,Top order,NaN,0.000000,0.196982,member
6,Marcus Stoinis,Australia,Allrounder,5.0,Middle order,Fast Medium,0.431579,0.819897,member
7,Tim David,Australia,Batter,6.0,Middle order,NaN,0.000000,0.974434,member
8,Ben Dwarshuis,Australia,Bowler,8.0,Finisher,Fast Medium,0.291149,0.386598,member
9,Xavier Bartlett,Australia,Bowler,8.0,Finisher,Fast Medium,0.490736,0.474420,member


In [579]:
venue_stats

,venue,avg_first_innings_score,avg_second_innings_score,avg_pp_runs,avg_death_runs,avg_wickets_per_match,chasing_win_pct
16,Arun Jaitley Stadium,175.000000,151.500000,22.500000,64.750000,10.000000,0.500000
98,Eden Gardens,142.285714,125.142857,22.428571,42.928571,12.285714,0.714286
210,"MA Chidambaram Stadium, Chepauk",150.500000,119.166667,25.500000,45.333333,12.166667,0.166667
246,Narendra Modi Stadium,170.600000,163.800000,27.600000,51.100000,10.400000,0.600000
271,Pallekele International Cricket Stadium,166.166667,146.533333,32.133333,49.736842,12.366667,0.400000
287,R Premadasa Stadium,141.696970,123.406250,25.153846,41.349206,12.757576,0.468750
337,Sinhalese Sports Club Ground,91.500000,98.000000,19.750000,33.333333,11.500000,1.000000
423,Wankhede Stadium,185.428571,177.000000,33.000000,60.714286,10.857143,0.714286


In [580]:
batting_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 19 to 2283
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player              279 non-null    object 
 1   team                279 non-null    object 
 2   balls_faced         279 non-null    int64  
 3   runs                279 non-null    int64  
 4   matches_played      279 non-null    int64  
 5   dismissals          279 non-null    int64  
 6   dot_balls           279 non-null    int64  
 7   fours               279 non-null    int64  
 8   sixes               279 non-null    int64  
 9   strike_rate         279 non-null    float64
 10  batting_average     279 non-null    float64
 11  dot_ball_pct        279 non-null    float64
 12  boundary_pct        279 non-null    float64
 13  batting_position    279 non-null    int64  
 14  death_strike_rate   279 non-null    float64
 15  middle_strike_rate  279 non-null    float64
 16  pp_strike_r

In [581]:
bowling_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 16 to 1641
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player               205 non-null    object 
 1   team                 205 non-null    object 
 2   balls_bowled         205 non-null    int64  
 3   runs_conceded        205 non-null    int64  
 4   matches_played       205 non-null    int64  
 5   dot_balls            205 non-null    int64  
 6   wickets              205 non-null    int64  
 7   overs_bowled         205 non-null    float64
 8   economy              205 non-null    float64
 9   dot_ball_pct         205 non-null    float64
 10  bowling_average      205 non-null    float64
 11  bowling_strike_rate  205 non-null    float64
 12  death_economy        205 non-null    float64
 13  middle_economy       205 non-null    float64
 14  pp_economy           205 non-null    float64
 15  bowling_role         205 non-null    object

In [582]:
player_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 208 to 239
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player              311 non-null    object 
 1   team                311 non-null    object 
 2   player_role         311 non-null    object 
 3   batting_position    309 non-null    float64
 4   batting_role        309 non-null    object 
 5   bowling_role        236 non-null    object 
 6   bowling_role_score  311 non-null    float64
 7   batting_role_score  309 non-null    float64
 8   role_in_team        311 non-null    object 
dtypes: float64(3), object(6)
memory usage: 24.3+ KB


### Team stats

In [583]:
bowling_stats[bowling_stats['team']=='India']

,player,team,balls_bowled,runs_conceded,matches_played,dot_balls,wickets,overs_bowled,economy,dot_ball_pct,bowling_average,bowling_strike_rate,death_economy,middle_economy,pp_economy,bowling_role
101,Axar Patel,India,545,686,30,215,37,90.833333,7.552294,0.394495,18.540541,14.729730,6.400000,7.216667,9.436364,Spin
138,Abhishek Sharma,India,155,225,17,43,6,25.833333,8.709677,0.277419,37.500000,25.833333,10.666667,7.834711,16.285714,Spin
201,Arshdeep Singh,India,694,942,30,279,55,115.666667,8.144092,0.402017,17.127273,12.618182,8.610879,5.731343,8.273196,Fast Medium
334,Varun Chakaravarthy,India,649,807,28,265,59,108.166667,7.460709,0.408320,13.677966,11.000000,6.835443,7.747178,6.850394,Spin
537,Hardik Pandya,India,623,853,34,235,35,103.833333,8.215088,0.377207,24.371429,17.800000,8.416667,8.608696,7.734940,Fast Medium
680,Jasprit Bumrah,India,503,579,22,232,33,83.833333,6.906561,0.461233,17.545455,15.242424,7.472393,6.880000,6.566038,Fast
819,Kuldeep Yadav,India,357,430,16,133,33,59.500000,7.226891,0.372549,13.030303,10.818182,5.454545,7.426710,10.000000,Spin
1002,Mohammed Siraj,India,123,119,6,62,3,20.500000,5.804878,0.504065,39.666667,41.000000,6.600000,4.666667,5.858824,Fast
1225,Rinku Singh,India,10,10,2,6,3,1.666667,6.000000,0.600000,3.333333,3.333333,6.000000,0.000000,0.000000,Part-time
1288,Shivam Dube,India,254,348,18,78,18,42.333333,8.220472,0.307087,19.333333,14.111111,9.846154,8.039409,6.000000,Medium Pace


In [584]:
import pandas as pd

# ---------------------------
# 1. Batting Team Aggregates
# ---------------------------
batting_team = batting_stats[batting_stats['batting_position']<=7].groupby('team').agg({
    'strike_rate': 'mean',
    'batting_average': 'mean',
    'boundary_pct': 'mean',
    'dot_ball_pct': 'mean',
    'pp_strike_rate': 'mean',
    'middle_strike_rate': 'mean',
    'death_strike_rate': 'mean'
}).reset_index()

batting_team.columns = [
    'team',
    'team_avg_strike_rate',
    'team_avg_batting_average',
    'team_avg_boundary_pct',
    'team_avg_dot_ball_pct',
    'pp_batting_strength',
    'middle_batting_strength',
    'death_batting_strength'
]

# ---------------------------
# 2. Bowling Team Aggregates
# ---------------------------
bowling_team = bowling_stats.groupby('team').agg({
    'economy': 'mean',
    'bowling_average': 'mean',
    'wickets': 'mean',
    'dot_ball_pct': 'mean',
    'pp_economy': 'mean',
    'middle_economy': 'mean',
    'death_economy': 'mean'
}).reset_index()

bowling_team.columns = [
    'team',
    'team_avg_economy',
    'team_avg_bowling_average',
    'team_avg_wickets',
    'team_dot_ball_pct_bowling',
    'pp_bowling_strength',
    'middle_bowling_strength',
    'death_bowling_strength'
]

# ---------------------------
# 4. Merge All into team_stats
# ---------------------------
team_stats = batting_team.merge(bowling_team, on='team', how='left')

In [585]:
team_stats['team_score'] = team_stats['team_avg_strike_rate'] *(1/ team_stats['team_avg_economy'])*team_stats['team_avg_batting_average']*team_stats['team_avg_boundary_pct']*(1/team_stats['team_avg_dot_ball_pct'])*(1/team_stats['team_avg_bowling_average'])*team_stats['team_avg_wickets']*team_stats['team_dot_ball_pct_bowling']

In [586]:
exp_team = ['India','Pakistan','England','New Zealand','Australia','South Africa','Sri Lanka']

In [587]:
exp_team = ['India', 'Pakistan', 'England', 'New Zealand', 'Australia', 'South Africa', 'Sri Lanka']

team_stats.loc[
    team_stats['team'].isin(exp_team),
    'team_score'
] *= 1.3


In [588]:
team_stats = team_stats.sort_values('team_score', ascending=False).reset_index(drop=True)
team_stats['rank'] = team_stats.index + 1

In [589]:
team_stats['team'].unique()

array(['India', 'Pakistan', 'Nepal', 'England', 'New Zealand',
       'United States of America', 'Canada', 'West Indies', 'Australia',
       'Zimbabwe', 'South Africa', 'Namibia', 'Sri Lanka',
       'United Arab Emirates', 'Ireland', 'Netherlands', 'Italy', 'Oman',
       'Scotland'], dtype=object)

In [590]:
team_stats

,team,team_avg_strike_rate,team_avg_batting_average,team_avg_boundary_pct,team_avg_dot_ball_pct,pp_batting_strength,middle_batting_strength,death_batting_strength,team_avg_economy,team_avg_bowling_average,team_avg_wickets,team_dot_ball_pct_bowling,pp_bowling_strength,middle_bowling_strength,death_bowling_strength,team_score,rank
0,India,155.050304,35.870821,0.225294,0.319882,126.292160,156.511719,171.426008,7.088196,18.108006,23.307692,0.409991,6.494127,5.966190,6.833939,379.126895,1
1,Pakistan,133.372826,24.278615,0.166280,0.324493,115.580917,132.114539,132.794076,7.310574,21.104054,24.900000,0.378471,7.162149,6.830438,7.266828,131.759627,2
2,Nepal,129.553394,25.339661,0.164328,0.350663,113.278815,111.064263,156.316296,7.388531,21.223159,24.400000,0.393029,5.348653,7.927227,6.653544,94.084552,3
3,England,138.504950,27.352169,0.184256,0.329290,121.144258,121.753046,137.034201,8.440050,20.798254,16.600000,0.358773,7.801106,8.413022,6.981448,93.497194,4
4,New Zealand,136.447482,27.072480,0.182933,0.328698,101.251143,143.369892,137.563886,8.604166,25.085203,18.909091,0.373999,5.622997,7.619395,8.728527,87.568165,5
5,United States of America,132.311210,32.340924,0.163796,0.313158,140.034480,118.528155,159.066353,7.338510,23.364933,17.625000,0.374090,6.395566,7.557950,7.157792,86.063896,6
6,Canada,121.249357,20.017785,0.147364,0.398358,60.386330,84.100659,145.851786,6.291873,12.092145,14.090909,0.451792,6.289374,4.702559,5.394024,75.128803,7
7,West Indies,136.104338,25.982594,0.189601,0.361910,126.019457,124.205823,116.514599,8.610902,22.964901,23.200000,0.303922,6.550119,8.846275,7.097787,66.058764,8
8,Australia,138.239176,26.255392,0.201299,0.387602,138.725316,115.997823,145.888233,8.241382,30.292223,18.181818,0.339326,8.316765,8.363404,7.800332,60.558061,9
9,Zimbabwe,131.028565,25.621022,0.171435,0.352413,98.541061,122.965276,167.519683,7.530062,22.748115,16.666667,0.374049,4.187993,7.492881,7.198580,59.435190,10


In [591]:
team_stats['team'].unique()

array(['India', 'Pakistan', 'Nepal', 'England', 'New Zealand',
       'United States of America', 'Canada', 'West Indies', 'Australia',
       'Zimbabwe', 'South Africa', 'Namibia', 'Sri Lanka',
       'United Arab Emirates', 'Ireland', 'Netherlands', 'Italy', 'Oman',
       'Scotland'], dtype=object)

In [592]:
player_data = player_data.sort_values(by=['team','batting_position'])

In [593]:
playing_xi_table = playing_xi_table.sort_values(by=['team','batting_position'])

In [594]:
playing_xi_table.to_csv("playing_xi_table.csv", index=False)

In [595]:
venue_stats.to_csv("venue_stats.csv", index=False)

In [599]:
team_stats.to_csv("team_stats.csv", index=False)

In [596]:
player_data[player_data['team']=='England']

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
234,Phil Salt,England,WicketKeeper,1.0,Opener,NaN,0.000000,0.806434,member
26,Ben Duckett,England,Batter,2.0,Opener,NaN,0.000000,0.746086,member
123,Jos Buttler,England,WicketKeeper,2.0,Opener,NaN,0.000000,0.795764,member
305,Will Jacks,England,Allrounder,2.0,Opener,Spin,0.626474,0.553041,member
105,Jacob Bethell,England,Allrounder,3.0,Top order,Spin,0.228026,0.539547,member
91,Harry Brook,England,Batter,4.0,Top order,NaN,0.000000,0.604975,captain
238,Rehan Ahmed,England,Bowler,4.0,Top order,Spin,0.190607,0.282039,member
258,Sam Curran,England,Allrounder,6.0,Middle order,Fast Medium,0.157051,0.475353,member
109,Jamie Overton,England,Allrounder,7.0,Finisher,Fast,0.416618,0.415820,member
46,Chris Jordan,England,Bowler,8.0,Finisher,Fast Medium,0.260120,0.117451,member


In [598]:
playing_xi_table[playing_xi_table['team']=='New Zealand']

,player,team,player_role,batting_position,batting_role,bowling_role,bowling_role_score,batting_role_score,role_in_team
99,Tim Seifert,New Zealand,WicketKeeper,1.0,Opener,NaN,0.000000,0.788922,member
100,Devon Conway,New Zealand,WicketKeeper,2.0,Opener,NaN,0.000000,0.623482,member
101,Finn Allen,New Zealand,Batter,2.0,Opener,NaN,0.000000,0.774485,member
102,Rachin Ravindra,New Zealand,Allrounder,3.0,Top order,Spin,0.294686,0.678245,member
103,Glenn Phillips,New Zealand,Batter,4.0,Top order,Spin,0.092385,0.641714,member
104,Mark Chapman,New Zealand,Batter,4.0,Top order,NaN,0.000000,0.708094,member
105,Daryl Mitchell,New Zealand,Allrounder,5.0,Middle order,Medium Pace,0.025208,0.739428,member
106,Michael Bracewell,New Zealand,Allrounder,7.0,Finisher,Spin,0.346637,0.587778,member
107,Mitchell Santner,New Zealand,Allrounder,7.0,Finisher,Spin,0.494741,0.688774,captain
108,Lockie Ferguson,New Zealand,Bowler,9.0,Tail,Fast,0.585794,0.000000,member
